# YOLO Model training

## Importations and initialisations

### Librairies

In [1]:
%load_ext autoreload
%autoreload 1

from google.colab import drive
drive.mount("/content/drive")
# Direct to your assignment folder.
%cd /content/drive/MyDrive/Cartonomics/Cocass

Mounted at /content/drive
/content/drive/MyDrive/Cartonomics/Cocass


In [2]:
!pip install -r requirements.txt

!pip install comet_ml sahi fiftyone ultralytics huggingface_hub funcy opencv-python albumentations tqdm pylabel


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.4/88.4 kB 5.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 5.3 MB/s eta 0:00:00
ERROR: Ignored the following versions that require a different python version: 0.6.0 Requires-Python >=3.5,<3.9; 0.6.0.1 Requires-Python >=3.5,<3.9; 0.6.1 Requires-Python >=3.5,<3.9; 0.6.1.1 Requires-Python >=3.5,<3.9; 0.6.2 Requires-Python >=3.5,<3.9; 0.6.3 Requires-Python >=3.5,<3.9; 0.6.4 Requires-Python >=3.5,<3.9; 0.6.5 Requires-Python >=3.5,<3.9; 0.6.5.1 Requires-Python >=3.5,<3.9; 0.6.5.2 Requires-Python >=3.5,<3.9; 0.6.6 Requires-Python >=3.5,<3.9; 0.7.0 Requires-Python >=3.5,<3.9
ERROR: Could not find a version that satisfies the requirement pywin32==306 (from versi

In [3]:
# Direct to your assignment folder.
%cd /content/drive/MyDrive/Cartonomics/Cocass/notebooks

from utils import init_notebook
%aimport datasets, datasets.data_transform
import datasets

import torch
import fiftyone as fo
from fiftyone import ViewField as F
from ultralytics import YOLO
import numpy as np
import os

import torch
import random
import cv2
from torch.utils.data import Dataset, DataLoader
from comet_ml import Experiment


from sahi import AutoDetectionModel
from sahi.predict import get_prediction, get_sliced_prediction

from pathlib import Path

HOME = Path(os.getcwd()).parents[0]
HOME


/content/drive/MyDrive/Cartonomics/Cocass/notebooks
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


PosixPath('/content/drive/MyDrive/Cartonomics/Cocass')

In [4]:
from ultralytics import YOLO
from ultralytics.data.augment import Albumentations
from ultralytics.utils import LOGGER, colorstr
from huggingface_hub import hf_hub_download

# Hugging Face repository info
model_repo = "crh-ehess/yolass-1.0"
model_filename = "yolass_1-0.pt"

# Download the model weights from Hugging Face
model_path = hf_hub_download(repo_id=model_repo, filename=model_filename)

# Load the model using the weights downloaded from Hugging Face
detection_model = YOLO(model_path)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


yolass_1-0.pt:   0%|          | 0.00/64.2M [00:00<?, ?B/s]

## Base Model importation
Optional it is now called later directlry in the training kernel

In [ ]:
from ultralytics import YOLO
from ultralytics.data.augment import Albumentations
from ultralytics.utils import LOGGER, colorstr
from huggingface_hub import hf_hub_download

def __init__(self, p=1.0):
    """
    Initialize the Albumentations transform object for YOLO bbox formatted parameters.

    This class applies various image augmentations using the Albumentations library, including Blur, Median Blur,
    conversion to grayscale, Contrast Limited Adaptive Histogram Equalization, random changes of brightness and
    contrast, RandomGamma, and image quality reduction through compression.

    Args:
        p (float): Probability of applying the augmentations. Must be between 0 and 1.

    Attributes:
            p (float): Probability of applying the augmentations.
            transform (albumentations.Compose): Composed Albumentations transforms.
            contains_spatial (bool): Indicates if the transforms include spatial transformations.

        Raises:
            ImportError: If the Albumentations package is not installed.
            Exception: For any other errors during initialization.

        Examples:
            >>> transform = Albumentations(p=0.5)
            >>> augmented = transform(image=image, bboxes=bboxes, class_labels=classes)
            >>> augmented_image = augmented["image"]
            >>> augmented_bboxes = augmented["bboxes"]

        Notes:
            - Requires Albumentations version 1.0.3 or higher.
            - Spatial transforms are handled differently to ensure bbox compatibility.
            - Some transforms are applied with very low probability (0.01) by default.
        """
    self.p = p
    self.transform = None
    prefix = colorstr("albumentations: ")

    try:
        import albumentations as A

        #check_version(A.__version__, "1.0.3", hard=True)  # version requirement

        # List of possible spatial transforms
        spatial_transforms = {
                "Affine",
                "BBoxSafeRandomCrop",
                "CenterCrop",
                "CoarseDropout",
                "Crop",
                "CropAndPad",
                "CropNonEmptyMaskIfExists",
                "D4",
                "ElasticTransform",
                "Flip",
                "GridDistortion",
                "GridDropout",
                "HorizontalFlip",
                "Lambda",
                "LongestMaxSize",
                "MaskDropout",
                "MixUp",
                "Morphological",
                "NoOp",
                "OpticalDistortion",
                "PadIfNeeded",
                "Perspective",
                "PiecewiseAffine",
                "PixelDropout",
                "RandomCrop",
                "RandomCropFromBorders",
                "RandomGridShuffle",
                "RandomResizedCrop",
                "RandomRotate90",
                "RandomScale",
                "RandomSizedBBoxSafeCrop",
                "RandomSizedCrop",
                "Resize",
                "Rotate",
                "SafeRotate",
                "ShiftScaleRotate",
                "SmallestMaxSize",
                "Transpose",
                "VerticalFlip",
                "XYMasking",
            }  # from https://albumentations.ai/docs/getting_started/transforms_and_targets/#spatial-level-transforms

            # Transforms
        T = [
                A.RandomCrop(height=768, width=768, p=1),
            ]

            # Compose transforms
        self.contains_spatial = any(transform.__class__.__name__ in spatial_transforms for transform in T)
        self.transform = (
                A.Compose(T, bbox_params=A.BboxParams(format="yolo", label_fields=["class_labels"]))
                if self.contains_spatial
                else A.Compose(T)
            )
        LOGGER.info(prefix + ", ".join(f"{x}".replace("always_apply=False, ", "") for x in T if x.p))
    except ImportError:  # package not installed, skip
        pass
    except Exception as e:
        LOGGER.info(f"{prefix}{e}")



#Albumentations.__init__ = __init__

# Hugging Face repository info
model_repo = "crh-ehess/yolass-1.0"
model_filename = "yolass_1-0.pt"

# Download the model weights from Hugging Face
model_path = hf_hub_download(repo_id=model_repo, filename=model_filename)

# Load the model using the weights downloaded from Hugging Face
detection_model = YOLO(model_path)

In [ ]:
from ultralytics import YOLO
from ultralytics.data.augment import Albumentations
from ultralytics.utils import LOGGER, colorstr
from huggingface_hub import hf_hub_download

# Hugging Face repository info
model_repo = "crh-ehess/yolass-1.0"
model_filename = "yolass_1-0.pt"

# Download the model weights from Hugging Face
model_path = hf_hub_download(repo_id=model_repo, filename=model_filename)

# Load the model using the weights downloaded from Hugging Face
detection_model = YOLO(model_path)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


yolass_1-0.pt:   0%|          | 0.00/64.2M [00:00<?, ?B/s]

## Data Importation

### Roboflow
Obsolete : Only if you need to import your datasets from Roboflow

In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key="<api-key>")
project = rf.workspace("<workspace>").project("cassini_maps")
version = project.version(2)
dataset = version.download("yolov8")

### Crop Augmentation

In [ ]:
import os
import cv2
import albumentations as A
from tqdm import tqdm
from pathlib import Path
import numpy as np

def get_random_crop_augmentation(height=640, width=640, p=1.0):
    """
    Returns an Albumentations augmentation pipeline with RandomCrop.

    Parameters:
    - height (int): The height of the crop.
    - width (int): The width of the crop.
    - p (float): Probability of applying the augmentation.

    Returns:
    - A.Compose: An Albumentations Compose object with the specified augmentations.
    """
    print("yes! Congratualations!\nRandom Crop seems to be working!\n")
    return A.Compose([
        A.RandomCrop(height=height, width=width, p=p),
    ],
    bbox_params=A.BboxParams(
        format='pascal_voc',  # Using 'pascal_voc' for absolute coordinates
        label_fields=['class_labels'],
        min_visibility=0.1  # Minimum visibility of bounding boxes after cropping
    ))

def convert_yolo_to_absolute(bbox, img_width, img_height):
    """
    Converts YOLO format to absolute pixel coordinates.

    Parameters:
    - bbox (list): [x_center, y_center, width, height] in YOLO format.
    - img_width (int): Width of the image.
    - img_height (int): Height of the image.

    Returns:
    - list: [x_min, y_min, x_max, y_max] in absolute coordinates.
    """
    x_center, y_center, width, height = bbox
    x_center *= img_width
    y_center *= img_height
    width *= img_width
    height *= img_height
    x_min = x_center - width / 2
    y_min = y_center - height / 2
    x_max = x_center + width / 2
    y_max = y_center + height / 2
    return [x_min, y_min, x_max, y_max]

def convert_absolute_to_yolo(bbox, img_width, img_height):
    """
    Converts absolute pixel coordinates to YOLO format.

    Parameters:
    - bbox (list): [x_min, y_min, x_max, y_max] in absolute coordinates.
    - img_width (int): Width of the image.
    - img_height (int): Height of the image.

    Returns:
    - list: [x_center, y_center, width, height] in YOLO format.
    """
    x_min, y_min, x_max, y_max = bbox
    x_center = ((x_min + x_max) / 2) / img_width
    y_center = ((y_min + y_max) / 2) / img_height
    width = (x_max - x_min) / img_width
    height = (y_max - y_min) / img_height
    # Clamp the values between 0.0 and 1.0
    x_center = np.clip(x_center, 0.0, 1.0)
    y_center = np.clip(y_center, 0.0, 1.0)
    width = np.clip(width, 0.0, 1.0)
    height = np.clip(height, 0.0, 1.0)
    return [x_center, y_center, width, height]

def augment_dataset(images_dir, labels_dir, augmented_images_dir, augmented_labels_dir, augmentation):
    """
    Applies augmentation to all images and labels in the dataset.

    Parameters:
    - images_dir (Path): Directory containing original images.
    - labels_dir (Path): Directory containing original labels.
    - augmented_images_dir (Path): Directory to save augmented images.
    - augmented_labels_dir (Path): Directory to save augmented labels.
    - augmentation (albumentations.Compose): The augmentation pipeline.
    """
    augmented_images_dir.mkdir(parents=True, exist_ok=True)
    augmented_labels_dir.mkdir(parents=True, exist_ok=True)

    image_files = [f for f in os.listdir(images_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp', '.tiff'))]

    for img_file in tqdm(image_files, desc="Augmenting images"):
        img_path = images_dir / img_file
        label_path = labels_dir / (img_file.rsplit('.', 1)[0] + '.txt')

        # Read image
        image = cv2.imread(str(img_path))
        if image is None:
            print(f"Warning: Unable to read image {img_path}. Skipping.")
            continue
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        img_height, img_width = image.shape[:2]

        # Read labels
        boxes = []
        labels = []
        if label_path.exists():
            with open(label_path, 'r') as f:
                for line_num, line in enumerate(f, 1):
                    parts = line.strip().split()
                    if len(parts) != 5:
                        print(f"Warning: Incorrect label format in {label_path} on line {line_num}. Skipping this label.")
                        continue
                    try:
                        # Convert class_id to float first, then to int
                        class_id = int(float(parts[0]))
                        bbox = list(map(float, parts[1:5]))
                        absolute_bbox = convert_yolo_to_absolute(bbox, img_width, img_height)
                        boxes.append(absolute_bbox)
                        labels.append(class_id)
                    except ValueError as ve:
                        print(f"Error: Unable to convert class ID '{parts[0]}' to int in {label_path} on line {line_num}. Skipping this label.")
                        continue
        else:
            print(f"Warning: Label file {label_path} does not exist. Skipping image {img_file}.")
            continue

        if len(boxes) == 0:
            print(f"Warning: No valid bounding boxes for image {img_file}. Skipping.")
            continue

        # Apply augmentation
        try:
            augmented = augmentation(image=image, bboxes=boxes, class_labels=labels)
        except Exception as e:
            print(f"Error during augmentation of image {img_file}: {e}. Skipping.")
            continue

        augmented_image = augmented['image']
        augmented_bboxes = augmented['bboxes']
        augmented_labels = augmented['class_labels']

        if len(augmented_bboxes) == 0:
            print(f"Info: No bounding boxes left after augmentation for image {img_file}. Skipping.")
            continue

        # Ensure augmented_bboxes have only 4 elements
        augmented_bboxes = [bbox for bbox in augmented_bboxes if len(bbox) == 4]

        if not augmented_bboxes:
            print(f"Info: All bounding boxes were removed after augmentation for image {img_file}. Skipping.")
            continue

        # Convert augmented image back to BGR for saving
        augmented_image_bgr = cv2.cvtColor(augmented_image, cv2.COLOR_RGB2BGR)

        # Save augmented image
        augmented_img_filename = f"aug_{img_file}"
        cv2.imwrite(str(augmented_images_dir / augmented_img_filename), augmented_image_bgr)

        # Convert augmented bounding boxes back to YOLO format
        augmented_yolo_bboxes = [convert_absolute_to_yolo(bbox, img_width, img_height) for bbox in augmented_bboxes]

        # Save augmented labels
        augmented_label_path = augmented_labels_dir / (augmented_img_filename.rsplit('.', 1)[0] + '.txt')
        with open(augmented_label_path, 'w') as f:
            for class_id, bbox in zip(augmented_labels, augmented_yolo_bboxes):
                bbox_str = ' '.join([f"{coord:.6f}" for coord in bbox])
                f.write(f"{class_id} {bbox_str}\n")


/usr/local/lib/python3.10/dist-packages/albumentations/__init__.py:24: UserWarning: A new version of Albumentations is available: 1.4.21 (you have 1.4.20). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [ ]:
from pathlib import Path

# Define paths

original_images_dir = HOME/"data/yolo_datasets/Detection/train/images"
original_labels_dir = HOME/"data/yolo_datasets/Detection/train/labels"
augmented_images_dir = HOME/"data/yolo_datasets/Detection/train_random_640/images"
augmented_labels_dir = HOME/"data/yolo_datasets/Detection/train_random_640/labels"

# Define augmentation
augmentation = get_random_crop_augmentation(height=640, width=640, p=1.0)

# Apply augmentation
augment_dataset(
    images_dir=original_images_dir,
    labels_dir=original_labels_dir,
    augmented_images_dir=augmented_images_dir,
    augmented_labels_dir=augmented_labels_dir,
    augmentation=augmentation
)


yes! Congratualations!
Random Crop seems to be working!



Augmenting images:   0%|          | 1/203 [00:00<02:51,  1.18it/s]

Augmenting images:   1%|          | 2/203 [00:03<06:11,  1.85s/it]

Augmenting images:   1%|▏         | 3/203 [00:05<06:07,  1.84s/it]

Augmenting images:   2%|▏         | 4/203 [00:06<05:32,  1.67s/it]

...

Augmenting images:  98%|█████████▊| 198/203 [02:13<00:03,  1.41it/s]

Augmenting images:  99%|█████████▉| 201/203 [02:14<00:00,  2.14it/s]

Error during augmentation of image 3786460736.png: Expected x_max for bbox [    0.98895      0.4849           1      0.5047           0] to be in the range [0.0, 1.0], got 1.0000499486923218.. Skipping.


Augmenting images: 100%|██████████| 203/203 [02:15<00:00,  1.50it/s]

Info: No bounding boxes left after augmentation for image 3787469173.png. Skipping.


### Model training

__If the training is not launching, check the paths set in your yaml file__

### Classify train_640 & val_640

In [5]:
# Here, comet is used to the logs during training and inspect if something went wrong. You can comment the following lines if you don't want to use comet (about experiment)
# The logs are also present in the runs folder
from comet_ml import Experiment
from comet_ml.integration.pytorch import log_model
torch.cuda.empty_cache()

experiment = Experiment(
  api_key="nHYScUpt7snJ6SYAs8tydMqcr",
  project_name="cartonomics",
  workspace="jiaming-jiang"
)

#Hyperparameters

results = detection_model.train(
    data=(HOME/"data/yolo_datasets/Classification/data.yaml").as_posix(),
    batch=8,  # Batch size
    epochs=50,  # Number of epochs
    close_mosaic=50,
    imgsz=640,  # Image size
    optimizer='AdamW',
    lr0=0.002,
    momentum=0.9,
    pretrained=True,
    project=(HOME/"runs/yolo/weighted_loss_2").as_posix(),  # Project directory
    name="weighted_loss_exp_2",
    cls=2.0,  # Increase the weight for classification loss
    patience=100,  # Early stopping patience
    verbose=False,  # Less verbose output
    cache=True,  # Cache dataset in memory
    amp=True,  # Enable mixed precision training
    hsv_h=0.0,
    hsv_s=0.0,
    hsv_v=0.0,
    translate=0.0,
    scale=0.0,
    mosaic=0.0,
    erasing=0.0,

)

# Save the fine-tuned YOLO model
detection_model.save(HOME/"outputs/train_640_classify.pt")

experiment.end()    #Do not forget to end the experiment to save the results in Comet
#this can take a while

COMET WARNING: To get all data logged automatically, import comet_ml before the following modules: sklearn, torch.
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/jiaming-jiang/cartonomics/52cc6e48db434905af0e00a6a7a166dd



Ultralytics 8.3.49 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=/root/.cache/huggingface/hub/models--crh-ehess--yolass-1.0/snapshots/df7779d835e9f045e21f3a29c85c4bdd7dfa74d4/yolass_1-0.pt, data=/content/drive/MyDrive/Cartonomics/Cocass/data/yolo_datasets/Classification/data.yaml, epochs=50, time=None, patience=100, batch=8, imgsz=640, save=True, save_period=-1, cache=True, device=None, workers=8, project=/content/drive/MyDrive/Cartonomics/Cocass/runs/yolo/weighted_loss_2, name=weighted_loss_exp_289, exist_ok=False, pretrained=True, optimizer=AdamW, verbose=False, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=50, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_str

100%|██████████| 755k/755k [00:00<00:00, 42.7MB/s]


Overriding model.yaml nc=17 with nc=13

                   from  n    params  module                                       arguments                     
  0                  -1  1      2320  ultralytics.nn.modules.conv.Conv             [3, 80, 3, 2]                 
  1                  -1  1    115520  ultralytics.nn.modules.conv.Conv             [80, 160, 3, 2]               
  2                  -1  3    436800  ultralytics.nn.modules.block.C2f             [160, 160, 3, True]           
  3                  -1  1    461440  ultralytics.nn.modules.conv.Conv             [160, 320, 3, 2]              
  4                  -1  6   3281920  ultralytics.nn.modules.block.C2f             [320, 320, 6, True]           
  5                  -1  1    213120  ultralytics.nn.modules.block.SCDown          [320, 640, 3, 2]              
  6                  -1  6   4604160  ultralytics.nn.modules.block.C2fCIB          [640, 640, 6, True]           
  7                  -1  1    417920  ultralytic

train: Scanning /content/drive/MyDrive/Cartonomics/Cocass/data/yolo_datasets/Classification/train/labels... 572 images, 228 backgrounds, 1 corrupt: 100%|██████████| 572/572 [00:39<00:00, 14.52it/s]

...

                   all        144       2899     0.0978     0.0889     0.0833     0.0404



50 epochs completed in 0.764 hours.
Optimizer stripped from /content/drive/MyDrive/Cartonomics/Cocass/runs/yolo/weighted_loss_2/weighted_loss_exp_289/weights/last.pt, 64.1MB
Optimizer stripped from /content/drive/MyDrive/Cartonomics/Cocass/runs/yolo/weighted_loss_2/weighted_loss_exp_289/weights/best.pt, 64.1MB

Validating /content/drive/MyDrive/Cartonomics/Cocass/runs/yolo/weighted_loss_2/weighted_loss_exp_289/weights/best.pt...
Ultralytics 8.3.49 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLOv10x summary (fused): 503 layers, 31,609,118 parameters, 0 gradients, 169.9 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 9/9 [00:05<00:00,  1.71it/s]


                   all        144       2899      0.411     0.0959     0.0965     0.0481
Speed: 0.2ms preprocess, 28.3ms inference, 0.0ms loss, 0.1ms postprocess per image
Results saved to /content/drive/MyDrive/Cartonomics/Cocass/runs/yolo/weighted_loss_2/weighted_loss_exp_289


COMET WARNING: Couldn't retrieve and log Google Colab notebook content, reason: 'NoneType' object is not subscriptable
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     name                  : physical_limestone_84
COMET INFO:     url                   : https://www.comet.com/jiaming-jiang/cartonomics/52cc6e48db434905af0e00a6a7a166dd
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     lr/pg0 [101]               : (5.9600000000000046e-05, 0.06778703703703703)
COMET INFO:     lr/pg1 [101]               : (5.9600000000000046e-05, 0.001911907407407407)
COMET INFO:     lr/pg2 [101]               : (5.9600000000000046e-05, 0.001911907407407407)
COMET INFO:     metrics/mAP50(B) [102]     : (0.03279, 0.10554)
COMET INFO:    

### Crop Trial

In [ ]:
# YOLODatasetWithCrop Class Definition
class YOLODatasetWithCrop(Dataset):
    def __init__(self, dataset, crop_size=(640, 640)):
        """
        Initialize the custom dataset with cropping.

        Args:
            dataset (Dataset): The original YOLO dataset instance.
            crop_size (tuple): Desired crop size (height, width).
        """
        self.dataset = dataset  # Original YOLO dataset
        self.crop_size = crop_size

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        """
        Get an item from the dataset with random cropping.

        Args:
            idx (int): Index of the image in the dataset.

        Returns:
            Tuple of cropped image and adjusted bounding boxes.
        """
        # Load image and targets from the original dataset
        img, targets = self.dataset[idx]  # Original image and YOLO-format targets
        h, w, _ = img.shape
        crop_h, crop_w = self.crop_size

        # Ensure crop size is within the bounds of the image
        if crop_h < h and crop_w < w:
            # Randomly choose the top-left corner of the crop
            top = random.randint(0, h - crop_h)
            left = random.randint(0, w - crop_w)

            # Crop the image
            img = img[top: top + crop_h, left: left + crop_w]

            # Adjust bounding boxes for the new crop
            new_targets = []
            for target in targets:
                cls, x_center, y_center, box_w, box_h = target  # YOLO format: [class, x_center, y_center, width, height]

                # Convert center-relative coordinates to absolute
                abs_x_center = x_center * w
                abs_y_center = y_center * h
                abs_box_w = box_w * w
                abs_box_h = box_h * h

                # Adjust to cropped coordinates
                crop_x_center = abs_x_center - left
                crop_y_center = abs_y_center - top

                # Ensure the box is within crop bounds
                if 0 <= crop_x_center < crop_w and 0 <= crop_y_center < crop_h:
                    # Normalize to cropped image size and store the new target
                    norm_x_center = crop_x_center / crop_w
                    norm_y_center = crop_y_center / crop_h
                    norm_box_w = abs_box_w / crop_w
                    norm_box_h = abs_box_h / crop_h
                    new_targets.append([cls, norm_x_center, norm_y_center, norm_box_w, norm_box_h])

            # Update targets to only include boxes within the cropped region
            targets = np.array(new_targets)

        return img, targets


In [ ]:
import yaml
# Here, comet is used to the logs during training and inspect if something went wrong. You can comment the following lines if you don't want to use comet (about experiment)
# The logs are also present in the runs folder
from comet_ml import Experiment
from comet_ml.integration.pytorch import log_model
torch.cuda.empty_cache()

# Comet Experiment Setup
experiment = Experiment(
    api_key="nHYScUpt7snJ6SYAs8tydMqcr",
    project_name="cartonomics",
    workspace="jiaming-jiang"
)

# Load data.yaml
data_yaml_path = Path("/content/drive/MyDrive/Cartonomics/Cocass/data/yolo_datasets/BlankDetection/data.yaml")
with open(data_yaml_path, "r") as f:
    data_config = yaml.safe_load(f)

# Define paths
train_img_dir = data_config['train']  # Path to training images directory
train_lbl_dir = os.path.join(os.path.dirname(train_img_dir), "../labels")


epochs = 50

# Define training parameters
train_params = {
    "data": data_yaml_path.as_posix(),  # Path to the data.yaml file
    "batch": 8,  # Batch size
    "epochs": epochs,  # Number of epochs
    #"imgsz": imgsz,  # Image size
    "plots": True,
    "pretrained": True,
    "project": (HOME / "runs/yolo/baseline").as_posix(),
    "name": f"yolass_1.0_{epochs}_ep_newbaseline",
    "save_dir": "train",
    "patience": 5,
    "verbose": False,
    "cache": True,
    "amp": True,  # Mixed precision training
    "cls": 1.0,  # Weight of classification loss
    "lr": 0.001,  # Learning rate
}


# Load the YOLO dataset manually
original_dataset = detection_model.dataset(data=data_yaml_path.as_posix(), split='train')

# Replace the dataset with the custom dataset that includes random cropping
custom_dataset = YOLODatasetWithCrop(original_dataset, crop_size=(640, 640))
train_loader = DataLoader(custom_dataset, batch_size=train_params["batch"], shuffle=True, num_workers=4)


# Start training with custom DataLoader
results = detection_model.train(
    **train_params,
    dataloader=train_loader  # Pass the custom DataLoader with cropping
)


# End Comet Experiment
experiment.end()

COMET WARNING: To get all data logged automatically, import comet_ml before the following modules: torch, sklearn.
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/jiaming-jiang/cartonomics/c2a1830c9ced4ad2b1655c1e06789e18



AttributeError: 'YOLO' object has no attribute 'dataset'

In [ ]:
# Here, comet is used to the logs during training and inspect if something went wrong. You can comment the following lines if you don't want to use comet (about experiment)
# The logs are also present in the runs folder
from comet_ml import Experiment
from comet_ml.integration.pytorch import log_model
torch.cuda.empty_cache()

experiment = Experiment(
  api_key="nHYScUpt7snJ6SYAs8tydMqcr",
  project_name="cartonomics",
  workspace="jiaming-jiang"
)

#experiment = Experiment(
#  api_key="<nHYScUpt7snJ6SYAs8tydMqcr>",    #your COMET API key
#  project_name="<project-name>",    # your project name
#  workspace="<username>"                   # your username
  #experiment_key = f"yolov10x_f52_{epochs}ep_imgsz1200_aug"
#)

#Hyperparameters
epochs = 60

imgsz = 768


results = detection_model.train(
    data=(HOME/"data/yolo_datasets/train/data.yaml").as_posix(),  # Path to the data.yaml file
    batch=8,  # Batch size
    epochs=epochs,  # Number of epochs
    imgsz=imgsz,  # Image size
    plots=True,
    pretrained=True,
    project=(HOME/"runs/yolo/baseline").as_posix(),  # Project directory
    name=f"yolass_1.0{epochs}_ep__{imgsz}imgs_train_newbaseline",
    save_dir="train",
    patience=5,  # Early stopping patience
    verbose=False,
    cache=True,  # Cache dataset
    amp=True,  # Mixed precision training
    cls=1.0,  # Weight of classification loss
    lr=0.001,  # Learning rate
    augment=True,  # Enable augmentations
    crop=True,  # Force cropping during training (if supported by the library)
    mosaic=False  # Disable mosaic to prioritize cropping
)

experiment.end()    #Do not forget to end the experiment to save the results in Comet
#this can take a while

COMET WARNING: To get all data logged automatically, import comet_ml before the following modules: torch, sklearn.
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/jiaming-jiang/cartonomics/fa6c461dc583462494f9842e5c4cde6a



SyntaxError: '[31m[1mcrop[0m' is not a valid YOLO argument. Similar arguments are i.e. ['save_crop=False'].
'[31m[1mlr[0m' is not a valid YOLO argument. Similar arguments are i.e. ['lrf=0.01', 'lr0=0.01'].

    Arguments received: ['yolo', '-f', '/root/.local/share/jupyter/runtime/kernel-0c880a2e-3017-41ca-b813-40dd57895581.json']. Ultralytics 'yolo' commands use the following syntax:

        yolo TASK MODE ARGS

        Where   TASK (optional) is one of {'pose', 'classify', 'detect', 'obb', 'segment'}
                MODE (required) is one of {'export', 'track', 'train', 'predict', 'val', 'benchmark'}
                ARGS (optional) are any number of custom 'arg=value' pairs like 'imgsz=320' that override defaults.
                    See all ARGS at https://docs.ultralytics.com/usage/cfg or with 'yolo cfg'

    1. Train a detection model for 10 epochs with an initial learning_rate of 0.01
        yolo train data=coco8.yaml model=yolo11n.pt epochs=10 lr0=0.01

    2. Predict a 

In [ ]:
import yaml

hyp_path = HOME/"data/yolo_datasets/BlankDetection/hyper.yaml"

# Load custom hyperparameters
with open(hyp_path, 'r') as f:
    custom_hyp = yaml.safe_load(f)

# Assign to the model
detection_model.model.hyp = custom_hyp

print(detection_model.model.hyp)


In [ ]:
import matplotlib.pyplot as plt
from ultralytics import YOLO
from huggingface_hub import hf_hub_download
from comet_ml import Experiment
from pathlib import Path
import yaml

# Hugging Face repository info
model_repo = "crh-ehess/yolass-1.0"
model_filename = "yolass_1-0.pt"

# Download the model weights from Hugging Face
model_path = hf_hub_download(repo_id=model_repo, filename=model_filename)

# Load the model using the weights downloaded from Hugging Face
detection_model = YOLO(model_path)

# Initialize Comet experiment
experiment = Experiment(
    api_key="nHYScUpt7snJ6SYAs8tydMqcr",
    project_name="cartonomics",
    workspace="jiaming-jiang"
)

# Path to the custom hyperparameters file
hyp_path = HOME/"data/yolo_datasets/BlankDetection/hyper.yaml"

# Load custom hyperparameters
with open(hyp_path, 'r') as f:
    custom_hyp = yaml.safe_load(f)

# Assign to the model
detection_model.model.hyp = custom_hyp

# Function to visualize augmented images
def visualize_augmentations(model, data_yaml, num_images=4):
    from ultralytics.yolo.utils import get_dataloader  # May not exist; adjust accordingly

    # Create data loader (this import may not work as per your error)
    # Hence, we'll use an alternative approach by manually applying augmentations
    import albumentations as A
    from PIL import Image
    import numpy as np

    # Define augmentation pipeline as per hyp_custom.yaml
    transform = A.Compose([
        A.RandomCrop(width=640, height=640, p=0.5),
        A.Blur(p=0.01),
        A.MedianBlur(blur_limit=7, p=0.01),
        A.ToGray(num_output_channels=3, p=0.01),
        A.CLAHE(clip_limit=4.0, tile_grid_size=(8, 8), p=0.01),
        # Add other augmentations as needed
    ])

    # Load dataset paths from data.yaml
    with open(data_yaml, 'r') as f:
        data_config = yaml.safe_load(f)

    train_images = Path(data_config['train']).glob('*/*.jpg')  # Adjust extension if needed

    # Visualize a few augmented images
    for idx, img_path in enumerate(train_images):
        if idx >= num_images:
            break
        image = np.array(Image.open(img_path).convert('RGB'))
        augmented = transform(image=image)
        augmented_image = augmented['image']

        plt.figure(figsize=(8,8))
        plt.imshow(augmented_image)
        plt.title(f"Augmented Image {idx+1}")
        plt.axis('off')
        plt.show()

visualize_augmentations(detection_model, HOME / "data/yolo_datasets/BlankDetection/data.yaml", num_images=4)




### PreCrop 768 (train_768 & valor_768)

In [ ]:
# Here, comet is used to the logs during training and inspect if something went wrong. You can comment the following lines if you don't want to use comet (about experiment)
# The logs are also present in the runs folder
from comet_ml import Experiment
from comet_ml.integration.pytorch import log_model
torch.cuda.empty_cache()

experiment = Experiment(
  api_key="nHYScUpt7snJ6SYAs8tydMqcr",
  project_name="cartonomics",
  workspace="jiaming-jiang"
)

#Hyperparameters

results = detection_model.train(
    data=(HOME/"data/yolo_datasets/Detection/data.yaml").as_posix(),
    batch=8,  # Batch size
    epochs=50,  # Number of epochs
    close_mosaic=50,
    imgsz=768,  # Image size
    optimizer='AdamW',
    lr0=0.002,
    momentum=0.9,
    pretrained=True,
    project=(HOME/"runs/yolo/weighted_loss_2").as_posix(),  # Project directory
    name="weighted_loss_exp_2",
    cls=2.0,  # Increase the weight for classification loss
    patience=100,  # Early stopping patience
    verbose=False,  # Less verbose output
    cache=True,  # Cache dataset in memory
    amp=True,  # Enable mixed precision training
    hsv_h=0.0,
    hsv_s=0.0,
    hsv_v=0.0,
    translate=0.0,
    scale=0.0,
    mosaic=0.0,
    erasing=0.0,

)

# Save the fine-tuned YOLO model
detection_model.save(HOME/"outputs/train_640.pt")

experiment.end()    #Do not forget to end the experiment to save the results in Comet
#this can take a while

COMET WARNING: To get all data logged automatically, import comet_ml before the following modules: sklearn, torch.
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/jiaming-jiang/cartonomics/ae0ffae6aa6048b4b98e2dd435aee6cf



Ultralytics 8.3.39 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=/root/.cache/huggingface/hub/models--crh-ehess--yolass-1.0/snapshots/df7779d835e9f045e21f3a29c85c4bdd7dfa74d4/yolass_1-0.pt, data=/content/drive/MyDrive/Cartonomics/Cocass/data/yolo_datasets/Detection/data.yaml, epochs=50, time=None, patience=100, batch=8, imgsz=768, save=True, save_period=-1, cache=True, device=None, workers=8, project=/content/drive/MyDrive/Cartonomics/Cocass/runs/yolo/weighted_loss_2, name=weighted_loss_exp_283, exist_ok=False, pretrained=True, optimizer=AdamW, verbose=False, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=50, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1

100%|██████████| 755k/755k [00:00<00:00, 39.6MB/s]


Overriding model.yaml nc=17 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1      2320  ultralytics.nn.modules.conv.Conv             [3, 80, 3, 2]                 
  1                  -1  1    115520  ultralytics.nn.modules.conv.Conv             [80, 160, 3, 2]               
  2                  -1  3    436800  ultralytics.nn.modules.block.C2f             [160, 160, 3, True]           
  3                  -1  1    461440  ultralytics.nn.modules.conv.Conv             [160, 320, 3, 2]              
  4                  -1  6   3281920  ultralytics.nn.modules.block.C2f             [320, 320, 6, True]           
  5                  -1  1    213120  ultralytics.nn.modules.block.SCDown          [320, 640, 3, 2]              
  6                  -1  6   4604160  ultralytics.nn.modules.block.C2fCIB          [640, 640, 6, True]           
  7                  -1  1    417920  ultralytics

train: Scanning /content/drive/MyDrive/Cartonomics/Cocass/data/yolo_datasets/Detection/train_768/labels.cache... 1181 images, 535 backgrounds, 16 corrupt: 100%|██████████| 1181/1181 [00:00<?, ?it/s]

...

                   all        192       4098      0.926      0.624      0.724      0.519



50 epochs completed in 2.119 hours.
Optimizer stripped from /content/drive/MyDrive/Cartonomics/Cocass/runs/yolo/weighted_loss_2/weighted_loss_exp_283/weights/last.pt, 64.1MB
Optimizer stripped from /content/drive/MyDrive/Cartonomics/Cocass/runs/yolo/weighted_loss_2/weighted_loss_exp_283/weights/best.pt, 64.1MB

Validating /content/drive/MyDrive/Cartonomics/Cocass/runs/yolo/weighted_loss_2/weighted_loss_exp_283/weights/best.pt...
Ultralytics 8.3.39 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLOv10x summary (fused): 503 layers, 31,586,006 parameters, 0 gradients, 169.8 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 12/12 [00:10<00:00,  1.13it/s]


                   all        192       4098      0.924      0.625      0.724       0.52
Speed: 0.3ms preprocess, 47.5ms inference, 0.0ms loss, 0.1ms postprocess per image
Results saved to /content/drive/MyDrive/Cartonomics/Cocass/runs/yolo/weighted_loss_2/weighted_loss_exp_283


COMET WARNING: Couldn't retrieve and log Google Colab notebook content, reason: 'NoneType' object is not subscriptable
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     name                  : tart_caulk_1306
COMET INFO:     url                   : https://www.comet.com/jiaming-jiang/cartonomics/ae0ffae6aa6048b4b98e2dd435aee6cf
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     lr/pg0 [101]               : (5.9600000000000046e-05, 0.06755707762557078)
COMET INFO:     lr/pg1 [101]               : (5.9600000000000046e-05, 0.0019164146118721458)
COMET INFO:     lr/pg2 [101]               : (5.9600000000000046e-05, 0.0019164146118721458)
COMET INFO:     metrics/mAP50(B) [102]     : (0.01924, 0.72609)
COMET INFO:     met

### PreCrop 768 (train_768 & valor_768)

In [ ]:
# Here, comet is used to the logs during training and inspect if something went wrong. You can comment the following lines if you don't want to use comet (about experiment)
# The logs are also present in the runs folder
from comet_ml import Experiment
from comet_ml.integration.pytorch import log_model
torch.cuda.empty_cache()

experiment = Experiment(
  api_key="nHYScUpt7snJ6SYAs8tydMqcr",
  project_name="cartonomics",
  workspace="jiaming-jiang"
)

#Hyperparameters

results = detection_model.train(
    data=(HOME/"data/yolo_datasets/Detection/data.yaml").as_posix(),
    batch=8,  # Batch size
    epochs=60,  # Number of epochs
    close_mosaic=60,
    imgsz=768,  # Image size
    optimizer='AdamW',
    lr0=0.002,
    momentum=0.9,
    pretrained=True,
    project=(HOME/"runs/yolo/train768&valor768").as_posix(),  # Project directory
    name="weighted_loss_exp_2",
    cls=2.0,  # Increase the weight for classification loss
    patience=100,  # Early stopping patience
    verbose=False,  # Less verbose output
    cache=True,  # Cache dataset in memory
    amp=True,  # Enable mixed precision training
    hsv_h=0.0,
    hsv_s=0.0,
    hsv_v=0.0,
    translate=0.0,
    scale=0.0,
    mosaic=0.0,
    erasing=0.0,

)

experiment.end()    #Do not forget to end the experiment to save the results in Comet
#this can take a while

COMET WARNING: To get all data logged automatically, import comet_ml before the following modules: torch, sklearn.
COMET WARNING: Couldn't retrieve and log Google Colab notebook content, reason: 'NoneType' object is not subscriptable
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     name                  : molecular_grapefruit_7417
COMET INFO:     url                   : https://www.comet.com/jiaming-jiang/cartonomics/2d3f268473cf4384a415b0583d256fc7
COMET INFO:   Others:
COMET INFO:     notebook_url : https://colab.research.google.com/notebook#fileId=1B1qLBVXMHTAz4_klqwxIs-fJzQaKpnBI
COMET INFO:   Uploads:
COMET INFO:     environment details      : 1
COMET INFO:     filename                 : 1
COMET INFO:     git metadat

Ultralytics 8.3.38 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (NVIDIA L4, 22700MiB)
engine/trainer: task=detect, mode=train, model=/root/.cache/huggingface/hub/models--crh-ehess--yolass-1.0/snapshots/df7779d835e9f045e21f3a29c85c4bdd7dfa74d4/yolass_1-0.pt, data=/content/drive/MyDrive/Cartonomics/Cocass/data/yolo_datasets/Detection/data.yaml, epochs=60, time=None, patience=100, batch=8, imgsz=768, save=True, save_period=-1, cache=True, device=None, workers=8, project=/content/drive/MyDrive/Cartonomics/Cocass/runs/yolo/train768&valor768, name=weighted_loss_exp_23, exist_ok=False, pretrained=True, optimizer=AdamW, verbose=False, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=60, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride

100%|██████████| 755k/755k [00:00<00:00, 112MB/s]


Overriding model.yaml nc=17 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1      2320  ultralytics.nn.modules.conv.Conv             [3, 80, 3, 2]                 
  1                  -1  1    115520  ultralytics.nn.modules.conv.Conv             [80, 160, 3, 2]               
  2                  -1  3    436800  ultralytics.nn.modules.block.C2f             [160, 160, 3, True]           
  3                  -1  1    461440  ultralytics.nn.modules.conv.Conv             [160, 320, 3, 2]              
  4                  -1  6   3281920  ultralytics.nn.modules.block.C2f             [320, 320, 6, True]           
  5                  -1  1    213120  ultralytics.nn.modules.block.SCDown          [320, 640, 3, 2]              
  6                  -1  6   4604160  ultralytics.nn.modules.block.C2fCIB          [640, 640, 6, True]           
  7                  -1  1    417920  ultralytics

train: Scanning /content/drive/MyDrive/Cartonomics/Cocass/data/yolo_datasets/Detection/train_768/labels.cache... 1181 images, 535 backgrounds, 16 corrupt: 100%|██████████| 1181/1181 [00:00<?, ?it/s]

...

                   all        192       4098      0.948      0.641      0.745      0.571



60 epochs completed in 1.461 hours.
Optimizer stripped from /content/drive/MyDrive/Cartonomics/Cocass/runs/yolo/train768&valor768/weighted_loss_exp_23/weights/last.pt, 64.1MB
Optimizer stripped from /content/drive/MyDrive/Cartonomics/Cocass/runs/yolo/train768&valor768/weighted_loss_exp_23/weights/best.pt, 64.1MB

Validating /content/drive/MyDrive/Cartonomics/Cocass/runs/yolo/train768&valor768/weighted_loss_exp_23/weights/best.pt...
Ultralytics 8.3.38 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (NVIDIA L4, 22700MiB)
YOLOv10x summary (fused): 503 layers, 31,586,006 parameters, 0 gradients, 169.8 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 12/12 [00:06<00:00,  1.92it/s]


                   all        192       4098      0.944      0.643      0.749      0.576
Speed: 0.2ms preprocess, 21.7ms inference, 0.0ms loss, 0.8ms postprocess per image
Results saved to /content/drive/MyDrive/Cartonomics/Cocass/runs/yolo/train768&valor768/weighted_loss_exp_23


COMET WARNING: Couldn't retrieve and log Google Colab notebook content, reason: 'NoneType' object is not subscriptable
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     name                  : horizontal_stadium_9530
COMET INFO:     url                   : https://www.comet.com/jiaming-jiang/cartonomics/2f15264ce1f347ae814891fddd4e3105
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     lr/pg0 [121]               : (5.30000000000001e-05, 0.06755707762557078)
COMET INFO:     lr/pg1 [121]               : (5.30000000000001e-05, 0.0019295844748858447)
COMET INFO:     lr/pg2 [121]               : (5.30000000000001e-05, 0.0019295844748858447)
COMET INFO:     metrics/mAP50(B) [122]     : (0.1069, 0.7492086971147065)
COMET I

### PreCrop 640 (train_640 & val_640)

In [ ]:
# Here, comet is used to the logs during training and inspect if something went wrong. You can comment the following lines if you don't want to use comet (about experiment)
# The logs are also present in the runs folder
from comet_ml import Experiment
from comet_ml.integration.pytorch import log_model
torch.cuda.empty_cache()

experiment = Experiment(
  api_key="nHYScUpt7snJ6SYAs8tydMqcr",
  project_name="cartonomics",
  workspace="jiaming-jiang"
)

#Hyperparameters

results = detection_model.train(
    data=(HOME/"data/yolo_datasets/Detection/data.yaml").as_posix(),
    batch=8,  # Batch size
    epochs=100,  # Number of epochs
    close_mosaic=100,
    imgsz=640,  # Image size
    optimizer='AdamW',
    lr0=0.002,
    momentum=0.9,
    pretrained=True,
    project=(HOME/"runs/yolo/weighted_loss_2").as_posix(),  # Project directory
    name="weighted_loss_exp_2",
    cls=2.0,  # Increase the weight for classification loss
    patience=100,  # Early stopping patience
    verbose=False,  # Less verbose output
    cache=True,  # Cache dataset in memory
    amp=True,  # Enable mixed precision training
    hsv_h=0.0,
    hsv_s=0.0,
    hsv_v=0.0,
    translate=0.0,
    scale=0.0,
    mosaic=0.0,
    erasing=0.0,

)

experiment.end()    #Do not forget to end the experiment to save the results in Comet
#this can take a while

COMET WARNING: To get all data logged automatically, import comet_ml before the following modules: sklearn, torch.
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/jiaming-jiang/cartonomics/0bfc71eeda314308853918c5140ee0be



Ultralytics 8.3.36 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (NVIDIA L4, 22700MiB)
engine/trainer: task=detect, mode=train, model=/root/.cache/huggingface/hub/models--crh-ehess--yolass-1.0/snapshots/df7779d835e9f045e21f3a29c85c4bdd7dfa74d4/yolass_1-0.pt, data=/content/drive/MyDrive/Cartonomics/Cocass/data/yolo_datasets/Detection/data.yaml, epochs=100, time=None, patience=100, batch=8, imgsz=640, save=True, save_period=-1, cache=True, device=None, workers=8, project=/content/drive/MyDrive/Cartonomics/Cocass/runs/yolo/weighted_loss_2, name=weighted_loss_exp_265, exist_ok=False, pretrained=True, optimizer=AdamW, verbose=False, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=100, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_strid

100%|██████████| 755k/755k [00:00<00:00, 20.0MB/s]


Overriding model.yaml nc=17 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1      2320  ultralytics.nn.modules.conv.Conv             [3, 80, 3, 2]                 
  1                  -1  1    115520  ultralytics.nn.modules.conv.Conv             [80, 160, 3, 2]               
  2                  -1  3    436800  ultralytics.nn.modules.block.C2f             [160, 160, 3, True]           
  3                  -1  1    461440  ultralytics.nn.modules.conv.Conv             [160, 320, 3, 2]              
  4                  -1  6   3281920  ultralytics.nn.modules.block.C2f             [320, 320, 6, True]           
  5                  -1  1    213120  ultralytics.nn.modules.block.SCDown          [320, 640, 3, 2]              
  6                  -1  6   4604160  ultralytics.nn.modules.block.C2fCIB          [640, 640, 6, True]           
  7                  -1  1    417920  ultralytics

train: Scanning /content/drive/MyDrive/Cartonomics/Cocass/data/yolo_datasets/Detection/train_640/labels... 2288 images, 1096 backgrounds, 0 corrupt: 100%|██████████| 2288/2288 [02:22<00:00, 16.08it/s]


...

                   all        576       6287      0.472      0.278      0.295      0.137



100 epochs completed in 3.236 hours.
Optimizer stripped from /content/drive/MyDrive/Cartonomics/Cocass/runs/yolo/weighted_loss_2/weighted_loss_exp_265/weights/last.pt, 64.1MB
Optimizer stripped from /content/drive/MyDrive/Cartonomics/Cocass/runs/yolo/weighted_loss_2/weighted_loss_exp_265/weights/best.pt, 64.1MB

Validating /content/drive/MyDrive/Cartonomics/Cocass/runs/yolo/weighted_loss_2/weighted_loss_exp_265/weights/best.pt...
Ultralytics 8.3.36 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (NVIDIA L4, 22700MiB)
YOLOv10x summary (fused): 503 layers, 31,586,006 parameters, 0 gradients, 169.8 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 36/36 [00:09<00:00,  3.87it/s]


                   all        576       6287       0.53      0.237      0.308      0.151
Speed: 0.2ms preprocess, 13.7ms inference, 0.0ms loss, 0.1ms postprocess per image
Results saved to /content/drive/MyDrive/Cartonomics/Cocass/runs/yolo/weighted_loss_2/weighted_loss_exp_265


COMET WARNING: Couldn't retrieve and log Google Colab notebook content, reason: 'NoneType' object is not subscriptable
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     name                  : identical_beetle_5299
COMET INFO:     url                   : https://www.comet.com/jiaming-jiang/cartonomics/0bfc71eeda314308853918c5140ee0be
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     lr/pg0 [201]               : (3.980000000000002e-05, 0.06744755244755245)
COMET INFO:     lr/pg1 [201]               : (3.980000000000002e-05, 0.001958115151515152)
COMET INFO:     lr/pg2 [201]               : (3.980000000000002e-05, 0.001958115151515152)
COMET INFO:     metrics/mAP50(B) [202]     : (0.13522, 0.3075339988043897)
COMET I

In [ ]:
# Here, comet is used to the logs during training and inspect if something went wrong. You can comment the following lines if you don't want to use comet (about experiment)
# The logs are also present in the runs folder
from comet_ml import Experiment
from comet_ml.integration.pytorch import log_model
torch.cuda.empty_cache()

experiment = Experiment(
  api_key="nHYScUpt7snJ6SYAs8tydMqcr",
  project_name="cartonomics",
  workspace="jiaming-jiang"
)

#Hyperparameters

results = detection_model.train(
    data=(HOME/"data/yolo_datasets/Detection/data.yaml").as_posix(),
    batch=8,  # Batch size
    epochs=50,  # Number of epochs
    close_mosaic=50,
    imgsz=640,  # Image size
    optimizer='AdamW',
    lr0=0.002,
    momentum=0.9,
    pretrained=True,
    project=(HOME/"runs/yolo/weighted_loss_2").as_posix(),  # Project directory
    name="weighted_loss_exp_2",
    cls=2.0,  # Increase the weight for classification loss
    patience=100,  # Early stopping patience
    verbose=False,  # Less verbose output
    cache=True,  # Cache dataset in memory
    amp=True,  # Enable mixed precision training
    hsv_h=0.0,
    hsv_s=0.0,
    hsv_v=0.0,
    translate=0.0,
    scale=0.0,
    mosaic=0.0,
    erasing=0.0,

)

# Save the fine-tuned YOLO model
detection_model.save(HOME/"outputs/train_640.pt")

experiment.end()    #Do not forget to end the experiment to save the results in Comet
#this can take a while

COMET WARNING: To get all data logged automatically, import comet_ml before the following modules: torch, sklearn.
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/jiaming-jiang/cartonomics/082a6d39f6624083b5e049066a20002e



Ultralytics 8.3.39 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=/root/.cache/huggingface/hub/models--crh-ehess--yolass-1.0/snapshots/df7779d835e9f045e21f3a29c85c4bdd7dfa74d4/yolass_1-0.pt, data=/content/drive/MyDrive/Cartonomics/Cocass/data/yolo_datasets/Detection/data.yaml, epochs=50, time=None, patience=100, batch=8, imgsz=640, save=True, save_period=-1, cache=True, device=None, workers=8, project=/content/drive/MyDrive/Cartonomics/Cocass/runs/yolo/weighted_loss_2, name=weighted_loss_exp_288, exist_ok=False, pretrained=True, optimizer=AdamW, verbose=False, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=50, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1

train: Scanning /content/drive/MyDrive/Cartonomics/Cocass/data/yolo_datasets/Detection/train_640/labels.cache... 2288 images, 1096 backgrounds, 0 corrupt: 100%|██████████| 2288/2288 [00:00<?, ?it/s]


WARNING ⚠️ cache='ram' may produce non-deterministic training results. Consider cache='disk' as a deterministic alternative if your disk space allows.


train: Caching images (2.6GB RAM): 100%|██████████| 2288/2288 [00:16<00:00, 141.36it/s]

...

                   all        340       6107      0.944      0.731       0.83      0.697



50 epochs completed in 2.724 hours.
Optimizer stripped from /content/drive/MyDrive/Cartonomics/Cocass/runs/yolo/weighted_loss_2/weighted_loss_exp_288/weights/last.pt, 64.1MB
Optimizer stripped from /content/drive/MyDrive/Cartonomics/Cocass/runs/yolo/weighted_loss_2/weighted_loss_exp_288/weights/best.pt, 64.1MB

Validating /content/drive/MyDrive/Cartonomics/Cocass/runs/yolo/weighted_loss_2/weighted_loss_exp_288/weights/best.pt...
Ultralytics 8.3.39 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLOv10x summary (fused): 503 layers, 31,586,006 parameters, 0 gradients, 169.8 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 22/22 [00:11<00:00,  1.83it/s]


                   all        340       6107      0.958      0.724      0.832      0.702
Speed: 0.2ms preprocess, 27.5ms inference, 0.0ms loss, 0.3ms postprocess per image
Results saved to /content/drive/MyDrive/Cartonomics/Cocass/runs/yolo/weighted_loss_2/weighted_loss_exp_288


COMET WARNING: Couldn't retrieve and log Google Colab notebook content, reason: 'NoneType' object is not subscriptable
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     name                  : respective_glazing_9067
COMET INFO:     url                   : https://www.comet.com/jiaming-jiang/cartonomics/082a6d39f6624083b5e049066a20002e
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     lr/pg0 [101]               : (5.9600000000000046e-05, 0.06744755244755245)
COMET INFO:     lr/pg1 [101]               : (5.9600000000000046e-05, 0.0019185613053613053)
COMET INFO:     lr/pg2 [101]               : (5.9600000000000046e-05, 0.0019185613053613053)
COMET INFO:     metrics/mAP50(B) [102]     : (0.67917, 0.83225)
COMET INFO:

###Precrop 640 (train_640 & val)

In [ ]:
# Here, comet is used to the logs during training and inspect if something went wrong. You can comment the following lines if you don't want to use comet (about experiment)
# The logs are also present in the runs folder
from comet_ml import Experiment
from comet_ml.integration.pytorch import log_model
torch.cuda.empty_cache()

experiment = Experiment(
  api_key="nHYScUpt7snJ6SYAs8tydMqcr",
  project_name="cartonomics",
  workspace="jiaming-jiang"
)

#Hyperparameters

results = detection_model.train(
    data=(HOME/"data/yolo_datasets/Detection/data.yaml").as_posix(),
    batch=8,  # Batch size
    epochs=50,  # Number of epochs
    close_mosaic=50,
    imgsz=640,  # Image size
    optimizer='AdamW',
    lr0=0.002,
    momentum=0.9,
    pretrained=True,
    project=(HOME/"runs/yolo/weighted_loss_2").as_posix(),  # Project directory
    name="weighted_loss_exp_2",
    cls=2.0,  # Increase the weight for classification loss
    patience=100,  # Early stopping patience
    verbose=False,  # Less verbose output
    cache=True,  # Cache dataset in memory
    amp=True,  # Enable mixed precision training
    hsv_h=0.0,
    hsv_s=0.0,
    hsv_v=0.0,
    translate=0.0,
    scale=0.0,
    mosaic=0.0,
    erasing=0.0,

)

experiment.end()    #Do not forget to end the experiment to save the results in Comet
#this can take a while

COMET WARNING: To get all data logged automatically, import comet_ml before the following modules: torch, sklearn.
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/jiaming-jiang/cartonomics/427d9c04e9c742e6b7a145c60fb85386



Ultralytics 8.3.37 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (NVIDIA L4, 22700MiB)
engine/trainer: task=detect, mode=train, model=/root/.cache/huggingface/hub/models--crh-ehess--yolass-1.0/snapshots/df7779d835e9f045e21f3a29c85c4bdd7dfa74d4/yolass_1-0.pt, data=/content/drive/MyDrive/Cartonomics/Cocass/data/yolo_datasets/Detection/data.yaml, epochs=50, time=None, patience=100, batch=8, imgsz=640, save=True, save_period=-1, cache=True, device=None, workers=8, project=/content/drive/MyDrive/Cartonomics/Cocass/runs/yolo/weighted_loss_2, name=weighted_loss_exp_267, exist_ok=False, pretrained=True, optimizer=AdamW, verbose=False, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=50, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=

train: Scanning /content/drive/MyDrive/Cartonomics/Cocass/data/yolo_datasets/Detection/train_640/labels.cache... 2288 images, 1096 backgrounds, 0 corrupt: 100%|██████████| 2288/2288 [00:00<?, ?it/s]


WARNING ⚠️ cache='ram' may produce non-deterministic training results. Consider cache='disk' as a deterministic alternative if your disk space allows.


train: Caching images (2.6GB RAM): 100%|██████████| 2288/2288 [00:10<00:00, 223.51it/s]

...

                   all        133       1596      0.591      0.311      0.379      0.193



50 epochs completed in 1.539 hours.
Optimizer stripped from /content/drive/MyDrive/Cartonomics/Cocass/runs/yolo/weighted_loss_2/weighted_loss_exp_267/weights/last.pt, 64.1MB
Optimizer stripped from /content/drive/MyDrive/Cartonomics/Cocass/runs/yolo/weighted_loss_2/weighted_loss_exp_267/weights/best.pt, 64.1MB

Validating /content/drive/MyDrive/Cartonomics/Cocass/runs/yolo/weighted_loss_2/weighted_loss_exp_267/weights/best.pt...
Ultralytics 8.3.37 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (NVIDIA L4, 22700MiB)
YOLOv10x summary (fused): 503 layers, 31,586,006 parameters, 0 gradients, 169.8 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 9/9 [00:02<00:00,  3.67it/s]


                   all        133       1596      0.655      0.281       0.39        0.2
Speed: 0.2ms preprocess, 14.0ms inference, 0.0ms loss, 1.0ms postprocess per image
Results saved to /content/drive/MyDrive/Cartonomics/Cocass/runs/yolo/weighted_loss_2/weighted_loss_exp_267


COMET WARNING: Couldn't retrieve and log Google Colab notebook content, reason: 'NoneType' object is not subscriptable
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     name                  : automatic_quasar_6473
COMET INFO:     url                   : https://www.comet.com/jiaming-jiang/cartonomics/427d9c04e9c742e6b7a145c60fb85386
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     lr/pg0 [101]               : (5.9600000000000046e-05, 0.06744755244755245)
COMET INFO:     lr/pg1 [101]               : (5.9600000000000046e-05, 0.0019185613053613053)
COMET INFO:     lr/pg2 [101]               : (5.9600000000000046e-05, 0.0019185613053613053)
COMET INFO:     metrics/mAP50(B) [102]     : (0.26472, 0.38992)
COMET INFO:  

###Precrop 512 (train_512 & val_512)

In [ ]:
# Here, comet is used to the logs during training and inspect if something went wrong. You can comment the following lines if you don't want to use comet (about experiment)
# The logs are also present in the runs folder
from comet_ml import Experiment
from comet_ml.integration.pytorch import log_model
torch.cuda.empty_cache()

experiment = Experiment(
  api_key="nHYScUpt7snJ6SYAs8tydMqcr",
  project_name="cartonomics",
  workspace="jiaming-jiang"
)

#Hyperparameters

results = detection_model.train(
    data=(HOME/"data/yolo_datasets/Detection/data.yaml").as_posix(),
    batch=16,  # Batch size
    epochs=50,  # Number of epochs
    close_mosaic=50,
    imgsz=512,  # Image size
    optimizer='AdamW',
    lr0=0.003,
    momentum=0.9,
    pretrained=True,
    project=(HOME/"runs/yolo/weighted_loss_2").as_posix(),  # Project directory
    name="weighted_loss_exp_2",
    cls=2.0,  # Increase the weight for classification loss
    patience=100,  # Early stopping patience
    verbose=False,  # Less verbose output
    cache=True,  # Cache dataset in memory
    amp=True,  # Enable mixed precision training
    hsv_h=0.0,
    hsv_s=0.0,
    hsv_v=0.0,
    translate=0.0,
    scale=0.0,
    mosaic=0.0,
    erasing=0.0,

)

# Save the fine-tuned YOLO model
detection_model.save(HOME/"outputs/train_512.pt")


experiment.end()    #Do not forget to end the experiment to save the results in Comet
#this can take a while

COMET WARNING: To get all data logged automatically, import comet_ml before the following modules: sklearn, torch.
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/jiaming-jiang/cartonomics/4cdbccf1b1c84cc381ace26756289a64



Ultralytics 8.3.39 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=/root/.cache/huggingface/hub/models--crh-ehess--yolass-1.0/snapshots/df7779d835e9f045e21f3a29c85c4bdd7dfa74d4/yolass_1-0.pt, data=/content/drive/MyDrive/Cartonomics/Cocass/data/yolo_datasets/Detection/data.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=512, save=True, save_period=-1, cache=True, device=None, workers=8, project=/content/drive/MyDrive/Cartonomics/Cocass/runs/yolo/weighted_loss_2, name=weighted_loss_exp_284, exist_ok=False, pretrained=True, optimizer=AdamW, verbose=False, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=50, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=

train: Scanning /content/drive/MyDrive/Cartonomics/Cocass/data/yolo_datasets/Detection/train_512/labels.cache... 2288 images, 1254 backgrounds, 0 corrupt: 100%|██████████| 2288/2288 [00:00<?, ?it/s]


WARNING ⚠️ cache='ram' may produce non-deterministic training results. Consider cache='disk' as a deterministic alternative if your disk space allows.


train: Caching images (1.7GB RAM): 100%|██████████| 2288/2288 [00:09<00:00, 231.30it/s]

...

                   all        576       4021      0.448       0.26      0.279      0.133



50 epochs completed in 1.875 hours.
Optimizer stripped from /content/drive/MyDrive/Cartonomics/Cocass/runs/yolo/weighted_loss_2/weighted_loss_exp_284/weights/last.pt, 64.1MB
Optimizer stripped from /content/drive/MyDrive/Cartonomics/Cocass/runs/yolo/weighted_loss_2/weighted_loss_exp_284/weights/best.pt, 64.1MB

Validating /content/drive/MyDrive/Cartonomics/Cocass/runs/yolo/weighted_loss_2/weighted_loss_exp_284/weights/best.pt...
Ultralytics 8.3.39 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLOv10x summary (fused): 503 layers, 31,586,006 parameters, 0 gradients, 169.8 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 18/18 [00:12<00:00,  1.40it/s]


                   all        576       4021      0.464      0.262      0.284      0.137
Speed: 0.1ms preprocess, 19.7ms inference, 0.0ms loss, 0.1ms postprocess per image
Results saved to /content/drive/MyDrive/Cartonomics/Cocass/runs/yolo/weighted_loss_2/weighted_loss_exp_284


COMET WARNING: Couldn't retrieve and log Google Colab notebook content, reason: 'NoneType' object is not subscriptable
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     name                  : continental_setback_1611
COMET INFO:     url                   : https://www.comet.com/jiaming-jiang/cartonomics/4cdbccf1b1c84cc381ace26756289a64
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     lr/pg0 [101]               : (8.940000000000006e-05, 0.0678927738927739)
COMET INFO:     lr/pg1 [101]               : (8.940000000000006e-05, 0.002874483916083916)
COMET INFO:     lr/pg2 [101]               : (8.940000000000006e-05, 0.002874483916083916)
COMET INFO:     metrics/mAP50(B) [102]     : (0.04018, 0.2891)
COMET INFO:     m

###Precrop 256 (train_256 & val_256)

In [ ]:
# Here, comet is used to the logs during training and inspect if something went wrong. You can comment the following lines if you don't want to use comet (about experiment)
# The logs are also present in the runs folder
from comet_ml import Experiment
from comet_ml.integration.pytorch import log_model
torch.cuda.empty_cache()

experiment = Experiment(
  api_key="nHYScUpt7snJ6SYAs8tydMqcr",
  project_name="cartonomics",
  workspace="jiaming-jiang"
)

#Hyperparameters

results = detection_model.train(
    data=(HOME/"data/yolo_datasets/Detection/data.yaml").as_posix(),
    batch=16,  # Batch size
    epochs=50,  # Number of epochs
    close_mosaic=50,
    imgsz=256,  # Image size
    optimizer='AdamW',
    lr0=0.002,
    momentum=0.9,
    pretrained=True,
    project=(HOME/"runs/yolo/weighted_loss_2").as_posix(),  # Project directory
    name="weighted_loss_exp_2",
    cls=2.0,  # Increase the weight for classification loss
    patience=100,  # Early stopping patience
    verbose=False,  # Less verbose output
    cache=True,  # Cache dataset in memory
    amp=True,  # Enable mixed precision training
    hsv_h=0.0,
    hsv_s=0.0,
    hsv_v=0.0,
    translate=0.0,
    scale=0.0,
    mosaic=0.0,
    erasing=0.0,

)

# Save the model weights manually after training
torch.save(detection_model.state_dict(), HOME/"outputs/train_256")


experiment.end()    #Do not forget to end the experiment to save the results in Comet
#this can take a while

COMET WARNING: To get all data logged automatically, import comet_ml before the following modules: torch, sklearn.
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/jiaming-jiang/cartonomics/e88b6ec4763f425db1d872206323c7cf



Ultralytics 8.3.39 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (NVIDIA L4, 22700MiB)
engine/trainer: task=detect, mode=train, model=/root/.cache/huggingface/hub/models--crh-ehess--yolass-1.0/snapshots/df7779d835e9f045e21f3a29c85c4bdd7dfa74d4/yolass_1-0.pt, data=/content/drive/MyDrive/Cartonomics/Cocass/data/yolo_datasets/Detection/data.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=256, save=True, save_period=-1, cache=True, device=None, workers=8, project=/content/drive/MyDrive/Cartonomics/Cocass/runs/yolo/weighted_loss_2, name=weighted_loss_exp_276, exist_ok=False, pretrained=True, optimizer=AdamW, verbose=False, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=50, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride

100%|██████████| 755k/755k [00:00<00:00, 19.4MB/s]


Overriding model.yaml nc=17 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1      2320  ultralytics.nn.modules.conv.Conv             [3, 80, 3, 2]                 
  1                  -1  1    115520  ultralytics.nn.modules.conv.Conv             [80, 160, 3, 2]               
  2                  -1  3    436800  ultralytics.nn.modules.block.C2f             [160, 160, 3, True]           
  3                  -1  1    461440  ultralytics.nn.modules.conv.Conv             [160, 320, 3, 2]              
  4                  -1  6   3281920  ultralytics.nn.modules.block.C2f             [320, 320, 6, True]           
  5                  -1  1    213120  ultralytics.nn.modules.block.SCDown          [320, 640, 3, 2]              
  6                  -1  6   4604160  ultralytics.nn.modules.block.C2fCIB          [640, 640, 6, True]           
  7                  -1  1    417920  ultralytics

train: Scanning /content/drive/MyDrive/Cartonomics/Cocass/data/yolo_datasets/Detection/train_256/labels.cache... 6569 images, 4834 backgrounds, 0 corrupt: 100%|██████████| 6569/6569 [00:00<?, ?it/s]


...

                   all       1604       2726      0.408       0.23      0.226      0.104



50 epochs completed in 1.203 hours.
Optimizer stripped from /content/drive/MyDrive/Cartonomics/Cocass/runs/yolo/weighted_loss_2/weighted_loss_exp_276/weights/last.pt, 64.0MB
Optimizer stripped from /content/drive/MyDrive/Cartonomics/Cocass/runs/yolo/weighted_loss_2/weighted_loss_exp_276/weights/best.pt, 64.0MB

Validating /content/drive/MyDrive/Cartonomics/Cocass/runs/yolo/weighted_loss_2/weighted_loss_exp_276/weights/best.pt...
Ultralytics 8.3.39 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (NVIDIA L4, 22700MiB)
YOLOv10x summary (fused): 503 layers, 31,586,006 parameters, 0 gradients, 169.8 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 51/51 [00:05<00:00, 10.12it/s]


                   all       1604       2726      0.407      0.263      0.256      0.115
Speed: 0.0ms preprocess, 2.0ms inference, 0.0ms loss, 0.1ms postprocess per image
Results saved to /content/drive/MyDrive/Cartonomics/Cocass/runs/yolo/weighted_loss_2/weighted_loss_exp_276


COMET WARNING: Couldn't retrieve and log Google Colab notebook content, reason: 'NoneType' object is not subscriptable
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     name                  : lonely_poutine_5675
COMET INFO:     url                   : https://www.comet.com/jiaming-jiang/cartonomics/e88b6ec4763f425db1d872206323c7cf
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     lr/pg0 [101]               : (5.9600000000000046e-05, 0.06741281427412815)
COMET INFO:     lr/pg1 [101]               : (5.9600000000000046e-05, 0.0019192421735604217)
COMET INFO:     lr/pg2 [101]               : (5.9600000000000046e-05, 0.0019192421735604217)
COMET INFO:     metrics/mAP50(B) [102]     : (0.04496, 0.25569629682168954)
COM

In [ ]:
# Here, comet is used to the logs during training and inspect if something went wrong. You can comment the following lines if you don't want to use comet (about experiment)
# The logs are also present in the runs folder
from comet_ml import Experiment
from comet_ml.integration.pytorch import log_model
torch.cuda.empty_cache()

experiment = Experiment(
  api_key="nHYScUpt7snJ6SYAs8tydMqcr",
  project_name="cartonomics",
  workspace="jiaming-jiang"
)

#Hyperparameters

results = detection_model.train(
    data=(HOME/"data/yolo_datasets/Detection/data.yaml").as_posix(),
    batch=16,  # Batch size
    epochs=50,  # Number of epochs
    close_mosaic=50,
    imgsz=256,  # Image size
    optimizer='AdamW',
    lr0=0.003,
    momentum=0.9,
    pretrained=True,
    project=(HOME/"runs/yolo/weighted_loss_2").as_posix(),  # Project directory
    name="weighted_loss_exp_2",
    cls=2.0,  # Increase the weight for classification loss
    patience=100,  # Early stopping patience
    verbose=False,  # Less verbose output
    cache=True,  # Cache dataset in memory
    amp=True,  # Enable mixed precision training
    hsv_h=0.0,
    hsv_s=0.0,
    hsv_v=0.0,
    translate=0.0,
    scale=0.0,
    mosaic=0.0,
    erasing=0.0,

)

# Save the fine-tuned YOLO model
detection_model.save(HOME/"outputs/train_256.pt")


experiment.end()    #Do not forget to end the experiment to save the results in Comet
#this can take a while

COMET WARNING: To get all data logged automatically, import comet_ml before the following modules: torch, sklearn.
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/jiaming-jiang/cartonomics/3bc180119b334771bc56b86d6e3c37ae



Ultralytics 8.3.39 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=/root/.cache/huggingface/hub/models--crh-ehess--yolass-1.0/snapshots/df7779d835e9f045e21f3a29c85c4bdd7dfa74d4/yolass_1-0.pt, data=/content/drive/MyDrive/Cartonomics/Cocass/data/yolo_datasets/Detection/data.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=256, save=True, save_period=-1, cache=True, device=None, workers=8, project=/content/drive/MyDrive/Cartonomics/Cocass/runs/yolo/weighted_loss_2, name=weighted_loss_exp_287, exist_ok=False, pretrained=True, optimizer=AdamW, verbose=False, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=50, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=

100%|██████████| 755k/755k [00:00<00:00, 86.7MB/s]


Overriding model.yaml nc=17 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1      2320  ultralytics.nn.modules.conv.Conv             [3, 80, 3, 2]                 
  1                  -1  1    115520  ultralytics.nn.modules.conv.Conv             [80, 160, 3, 2]               
  2                  -1  3    436800  ultralytics.nn.modules.block.C2f             [160, 160, 3, True]           
  3                  -1  1    461440  ultralytics.nn.modules.conv.Conv             [160, 320, 3, 2]              
  4                  -1  6   3281920  ultralytics.nn.modules.block.C2f             [320, 320, 6, True]           
  5                  -1  1    213120  ultralytics.nn.modules.block.SCDown          [320, 640, 3, 2]              
  6                  -1  6   4604160  ultralytics.nn.modules.block.C2fCIB          [640, 640, 6, True]           
  7                  -1  1    417920  ultralytics

train: Scanning /content/drive/MyDrive/Cartonomics/Cocass/data/yolo_datasets/Detection/train_256/labels.cache... 6569 images, 4834 backgrounds, 0 corrupt: 100%|██████████| 6569/6569 [00:00<?, ?it/s]


...

                   all       1604       2726       0.38      0.248      0.228      0.102



50 epochs completed in 1.554 hours.
Optimizer stripped from /content/drive/MyDrive/Cartonomics/Cocass/runs/yolo/weighted_loss_2/weighted_loss_exp_287/weights/last.pt, 64.0MB
Optimizer stripped from /content/drive/MyDrive/Cartonomics/Cocass/runs/yolo/weighted_loss_2/weighted_loss_exp_287/weights/best.pt, 64.0MB

Validating /content/drive/MyDrive/Cartonomics/Cocass/runs/yolo/weighted_loss_2/weighted_loss_exp_287/weights/best.pt...
Ultralytics 8.3.39 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLOv10x summary (fused): 503 layers, 31,586,006 parameters, 0 gradients, 169.8 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 51/51 [00:10<00:00,  5.10it/s]


                   all       1604       2726      0.358      0.277       0.24      0.106
Speed: 0.0ms preprocess, 5.0ms inference, 0.0ms loss, 0.2ms postprocess per image
Results saved to /content/drive/MyDrive/Cartonomics/Cocass/runs/yolo/weighted_loss_2/weighted_loss_exp_287


COMET WARNING: Couldn't retrieve and log Google Colab notebook content, reason: 'NoneType' object is not subscriptable
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     name                  : fierce_gable_6216
COMET INFO:     url                   : https://www.comet.com/jiaming-jiang/cartonomics/3bc180119b334771bc56b86d6e3c37ae
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     lr/pg0 [101]               : (8.940000000000006e-05, 0.06774533657745337)
COMET INFO:     lr/pg1 [101]               : (8.940000000000006e-05, 0.002878863260340633)
COMET INFO:     lr/pg2 [101]               : (8.940000000000006e-05, 0.002878863260340633)
COMET INFO:     metrics/mAP50(B) [102]     : (0.03678, 0.24013)
COMET INFO:     metric

###Precrop 640 Random (train+train_random_640 & val)

In [ ]:
# Here, comet is used to the logs during training and inspect if something went wrong. You can comment the following lines if you don't want to use comet (about experiment)
# The logs are also present in the runs folder
from comet_ml import Experiment
from comet_ml.integration.pytorch import log_model
torch.cuda.empty_cache()

experiment = Experiment(
  api_key="nHYScUpt7snJ6SYAs8tydMqcr",
  project_name="cartonomics",
  workspace="jiaming-jiang"
)

#Hyperparameters

results = detection_model.train(
    data=(HOME/"data/yolo_datasets/Detection/data.yaml").as_posix(),
    batch=8,  # Batch size
    epochs=50,  # Number of epochs
    close_mosaic=50,
    imgsz=640,  # Image size
    optimizer='AdamW',
    lr0=0.002,
    momentum=0.9,
    pretrained=True,
    project=(HOME/"runs/yolo/weighted_loss_2").as_posix(),  # Project directory
    name="weighted_loss_exp_2",
    cls=2.0,  # Increase the weight for classification loss
    patience=100,  # Early stopping patience
    verbose=False,  # Less verbose output
    cache=True,  # Cache dataset in memory
    amp=True,  # Enable mixed precision training
    hsv_h=0.0,
    hsv_s=0.0,
    hsv_v=0.0,
    translate=0.0,
    scale=0.0,
    mosaic=0.0,
    erasing=0.0,

)

experiment.end()    #Do not forget to end the experiment to save the results in Comet
#this can take a while

COMET WARNING: To get all data logged automatically, import comet_ml before the following modules: sklearn, torch.
COMET WARNING: Couldn't retrieve and log Google Colab notebook content, reason: 'NoneType' object is not subscriptable
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     name                  : significant_blackberry_7040
COMET INFO:     url                   : https://www.comet.com/jiaming-jiang/cartonomics/b27a3238cf884358844de94c8fc8b1ae
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     lr/pg0 [28]               : (0.0014852, 0.06777477477477478)
COMET INFO:     lr/pg1 [28]               : (0.0006576576576576577, 0.0019121477477477475)
COMET INFO:     lr/pg2 [28]               : (0.000657657657657657

Ultralytics 8.3.37 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (NVIDIA L4, 22700MiB)
engine/trainer: task=detect, mode=train, model=/root/.cache/huggingface/hub/models--crh-ehess--yolass-1.0/snapshots/df7779d835e9f045e21f3a29c85c4bdd7dfa74d4/yolass_1-0.pt, data=/content/drive/MyDrive/Cartonomics/Cocass/data/yolo_datasets/Detection/data.yaml, epochs=50, time=None, patience=100, batch=8, imgsz=640, save=True, save_period=-1, cache=True, device=None, workers=8, project=/content/drive/MyDrive/Cartonomics/Cocass/runs/yolo/weighted_loss_2, name=weighted_loss_exp_270, exist_ok=False, pretrained=True, optimizer=AdamW, verbose=False, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=50, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=

train: Scanning /content/drive/MyDrive/Cartonomics/Cocass/data/yolo_datasets/Detection/train/labels.cache... 640 images, 226 backgrounds, 50 corrupt: 100%|██████████| 640/640 [00:00<?, ?it/s]

train: WARNING ⚠️ /content/drive/MyDrive/Cartonomics/Cocass/data/yolo_datasets/Detection/train/images/3785973075.png: ignoring corrupt image/label: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (38,) + inhomogeneous part.
train: WARNING ⚠️ /content/drive/MyDrive/Cartonomics/Cocass/data/yolo_datasets/Detection/train/images/3786132335.png: ignoring corrupt image/label: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (71,) + inhomogeneous part.
train: WARNING ⚠️ /content/drive/MyDrive/Cartonomics/Cocass/data/yolo_datasets/Detection/train/images/3786135226.png: ignoring corrupt image/label: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (64,) + inhomogeneous part.
train: WARNING ⚠️ /content/drive/MyDrive/Cartonomics/Cocass/data/yolo_datase

WARNING ⚠️ cache='ram' may produce non-deterministic training results. Consider cache='disk' as a deterministic alternative if your disk space allows.


...

                   all        133       1596      0.592      0.297      0.317      0.155



50 epochs completed in 0.449 hours.
Optimizer stripped from /content/drive/MyDrive/Cartonomics/Cocass/runs/yolo/weighted_loss_2/weighted_loss_exp_270/weights/last.pt, 64.1MB
Optimizer stripped from /content/drive/MyDrive/Cartonomics/Cocass/runs/yolo/weighted_loss_2/weighted_loss_exp_270/weights/best.pt, 64.1MB

Validating /content/drive/MyDrive/Cartonomics/Cocass/runs/yolo/weighted_loss_2/weighted_loss_exp_270/weights/best.pt...
Ultralytics 8.3.37 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (NVIDIA L4, 22700MiB)
YOLOv10x summary (fused): 503 layers, 31,586,006 parameters, 0 gradients, 169.8 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 9/9 [00:02<00:00,  3.26it/s]


                   all        133       1596      0.539      0.332      0.351      0.169
Speed: 0.2ms preprocess, 13.9ms inference, 0.0ms loss, 0.9ms postprocess per image
Results saved to /content/drive/MyDrive/Cartonomics/Cocass/runs/yolo/weighted_loss_2/weighted_loss_exp_270


COMET WARNING: Couldn't retrieve and log Google Colab notebook content, reason: 'NoneType' object is not subscriptable
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     name                  : olive_rasp_9678
COMET INFO:     url                   : https://www.comet.com/jiaming-jiang/cartonomics/568ec9142f6a401d861278b2a1aff42d
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     lr/pg0 [101]               : (5.9600000000000046e-05, 0.06777477477477478)
COMET INFO:     lr/pg1 [101]               : (5.9600000000000046e-05, 0.0019121477477477475)
COMET INFO:     lr/pg2 [101]               : (5.9600000000000046e-05, 0.0019121477477477475)
COMET INFO:     metrics/mAP50(B) [102]     : (0.23943, 0.3507385152829945)
COMET IN

### Precrop 640 (train+train_random_640 & val_original)

In [ ]:
# Here, comet is used to the logs during training and inspect if something went wrong. You can comment the following lines if you don't want to use comet (about experiment)
# The logs are also present in the runs folder
from comet_ml import Experiment
from comet_ml.integration.pytorch import log_model
torch.cuda.empty_cache()

experiment = Experiment(
  api_key="nHYScUpt7snJ6SYAs8tydMqcr",
  project_name="cartonomics",
  workspace="jiaming-jiang"
)

#Hyperparameters

results = detection_model.train(
    data=(HOME/"data/yolo_datasets/Detection/data.yaml").as_posix(),
    batch=8,  # Batch size
    epochs=50,  # Number of epochs
    close_mosaic=50,
    imgsz=640,  # Image size
    optimizer='AdamW',
    lr0=0.002,
    momentum=0.9,
    pretrained=True,
    project=(HOME/"runs/yolo/weighted_loss_2").as_posix(),  # Project directory
    name="weighted_loss_exp_2",
    cls=2.0,  # Increase the weight for classification loss
    patience=100,  # Early stopping patience
    verbose=False,  # Less verbose output
    cache=True,  # Cache dataset in memory
    amp=True,  # Enable mixed precision training
    hsv_h=0.0,
    hsv_s=0.0,
    hsv_v=0.0,
    translate=0.0,
    scale=0.0,
    mosaic=0.0,
    erasing=0.0,

)

experiment.end()    #Do not forget to end the experiment to save the results in Comet
#this can take a while

COMET WARNING: To get all data logged automatically, import comet_ml before the following modules: sklearn, torch.
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/jiaming-jiang/cartonomics/bbff1b2c42bd4e6fa0d4258336fcc604



Ultralytics 8.3.37 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (NVIDIA L4, 22700MiB)
engine/trainer: task=detect, mode=train, model=/root/.cache/huggingface/hub/models--crh-ehess--yolass-1.0/snapshots/df7779d835e9f045e21f3a29c85c4bdd7dfa74d4/yolass_1-0.pt, data=/content/drive/MyDrive/Cartonomics/Cocass/data/yolo_datasets/Detection/data.yaml, epochs=50, time=None, patience=100, batch=8, imgsz=640, save=True, save_period=-1, cache=True, device=None, workers=8, project=/content/drive/MyDrive/Cartonomics/Cocass/runs/yolo/weighted_loss_2, name=weighted_loss_exp_271, exist_ok=False, pretrained=True, optimizer=AdamW, verbose=False, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=50, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=

train: Scanning /content/drive/MyDrive/Cartonomics/Cocass/data/yolo_datasets/Detection/train/labels.cache... 640 images, 226 backgrounds, 50 corrupt: 100%|██████████| 640/640 [00:00<?, ?it/s]

train: WARNING ⚠️ /content/drive/MyDrive/Cartonomics/Cocass/data/yolo_datasets/Detection/train/images/3785973075.png: ignoring corrupt image/label: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (38,) + inhomogeneous part.
train: WARNING ⚠️ /content/drive/MyDrive/Cartonomics/Cocass/data/yolo_datasets/Detection/train/images/3786132335.png: ignoring corrupt image/label: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (71,) + inhomogeneous part.
train: WARNING ⚠️ /content/drive/MyDrive/Cartonomics/Cocass/data/yolo_datasets/Detection/train/images/3786135226.png: ignoring corrupt image/label: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (64,) + inhomogeneous part.
train: WARNING ⚠️ /content/drive/MyDrive/Cartonomics/Cocass/data/yolo_datase

WARNING ⚠️ cache='ram' may produce non-deterministic training results. Consider cache='disk' as a deterministic alternative if your disk space allows.


...

                   all         69       1532      0.917      0.656      0.749       0.54



50 epochs completed in 0.451 hours.
Optimizer stripped from /content/drive/MyDrive/Cartonomics/Cocass/runs/yolo/weighted_loss_2/weighted_loss_exp_271/weights/last.pt, 64.1MB
Optimizer stripped from /content/drive/MyDrive/Cartonomics/Cocass/runs/yolo/weighted_loss_2/weighted_loss_exp_271/weights/best.pt, 64.1MB

Validating /content/drive/MyDrive/Cartonomics/Cocass/runs/yolo/weighted_loss_2/weighted_loss_exp_271/weights/best.pt...
Ultralytics 8.3.37 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (NVIDIA L4, 22700MiB)
YOLOv10x summary (fused): 503 layers, 31,586,006 parameters, 0 gradients, 169.8 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:02<00:00,  1.74it/s]


                   all         69       1532      0.917      0.656      0.749       0.54
Speed: 0.3ms preprocess, 13.4ms inference, 0.0ms loss, 0.9ms postprocess per image
Results saved to /content/drive/MyDrive/Cartonomics/Cocass/runs/yolo/weighted_loss_2/weighted_loss_exp_271


COMET WARNING: Couldn't retrieve and log Google Colab notebook content, reason: 'NoneType' object is not subscriptable
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     name                  : poised_ply_7681
COMET INFO:     url                   : https://www.comet.com/jiaming-jiang/cartonomics/bbff1b2c42bd4e6fa0d4258336fcc604
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     lr/pg0 [101]               : (5.9600000000000046e-05, 0.06777477477477478)
COMET INFO:     lr/pg1 [101]               : (5.9600000000000046e-05, 0.0019121477477477475)
COMET INFO:     lr/pg2 [101]               : (5.9600000000000046e-05, 0.0019121477477477475)
COMET INFO:     metrics/mAP50(B) [102]     : (0.43785, 0.75145)
COMET INFO:     met

### Epoch100Batch8 Blanked Cropped 640 - SingleLabel

In [ ]:
# Here, comet is used to the logs during training and inspect if something went wrong. You can comment the following lines if you don't want to use comet (about experiment)
# The logs are also present in the runs folder
from comet_ml import Experiment
from comet_ml.integration.pytorch import log_model
torch.cuda.empty_cache()

experiment = Experiment(
  api_key="nHYScUpt7snJ6SYAs8tydMqcr",
  project_name="cartonomics",
  workspace="jiaming-jiang"
)

#Hyperparameters

results = detection_model.train(
    data=(HOME/"data/yolo_datasets/Detection/data.yaml").as_posix(),
    batch=8,  # Batch size
    epochs=100,  # Number of epochs
    close_mosaic=100,
    imgsz=640,  # Image size
    optimizer='Adam',
    lr0=0.002,
    momentum=0.9,
    pretrained=True,
    project=(HOME/"runs/yolo/weighted_loss_2").as_posix(),  # Project directory
    name="weighted_loss_exp_2",
    cls=2.0,  # Increase the weight for classification loss
    patience=100,  # Early stopping patience
    verbose=False,  # Less verbose output
    cache=True,  # Cache dataset in memory
    amp=True,  # Enable mixed precision training
    #augment=False,
    #hyp=hyp_path.as_posix(),  # Pass the custom hyperparameters file
)

experiment.end()    #Do not forget to end the experiment to save the results in Comet
#this can take a while

COMET WARNING: To get all data logged automatically, import comet_ml before the following modules: sklearn, torch.
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/jiaming-jiang/cartonomics/6a246ebd3c1948cc8e18e7e5d8e5a6a9



Ultralytics 8.3.32 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (NVIDIA L4, 22700MiB)
engine/trainer: task=detect, mode=train, model=/root/.cache/huggingface/hub/models--crh-ehess--yolass-1.0/snapshots/df7779d835e9f045e21f3a29c85c4bdd7dfa74d4/yolass_1-0.pt, data=/content/drive/MyDrive/Cartonomics/Cocass/data/yolo_datasets/Detection/data.yaml, epochs=100, time=None, patience=100, batch=8, imgsz=640, save=True, save_period=-1, cache=True, device=None, workers=8, project=/content/drive/MyDrive/Cartonomics/Cocass/runs/yolo/weighted_loss_2, name=weighted_loss_exp_254, exist_ok=False, pretrained=True, optimizer=Adam, verbose=False, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=100, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride

100%|██████████| 755k/755k [00:00<00:00, 123MB/s]


Overriding model.yaml nc=17 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1      2320  ultralytics.nn.modules.conv.Conv             [3, 80, 3, 2]                 
  1                  -1  1    115520  ultralytics.nn.modules.conv.Conv             [80, 160, 3, 2]               
  2                  -1  3    436800  ultralytics.nn.modules.block.C2f             [160, 160, 3, True]           
  3                  -1  1    461440  ultralytics.nn.modules.conv.Conv             [160, 320, 3, 2]              
  4                  -1  6   3281920  ultralytics.nn.modules.block.C2f             [320, 320, 6, True]           
  5                  -1  1    213120  ultralytics.nn.modules.block.SCDown          [320, 640, 3, 2]              
  6                  -1  6   4604160  ultralytics.nn.modules.block.C2fCIB          [640, 640, 6, True]           
  7                  -1  1    417920  ultralytics

train: Scanning /content/drive/MyDrive/Cartonomics/Cocass/data/yolo_datasets/Detection/train/labels.cache... 572 images, 226 backgrounds, 50 corrupt: 100%|██████████| 572/572 [00:00<?, ?it/s]

...

                   all        133       1596      0.578      0.469      0.456      0.236



100 epochs completed in 0.798 hours.
Optimizer stripped from /content/drive/MyDrive/Cartonomics/Cocass/runs/yolo/weighted_loss_2/weighted_loss_exp_254/weights/last.pt, 64.1MB
Optimizer stripped from /content/drive/MyDrive/Cartonomics/Cocass/runs/yolo/weighted_loss_2/weighted_loss_exp_254/weights/best.pt, 64.1MB

Validating /content/drive/MyDrive/Cartonomics/Cocass/runs/yolo/weighted_loss_2/weighted_loss_exp_254/weights/best.pt...
Ultralytics 8.3.32 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (NVIDIA L4, 22700MiB)
YOLOv10x summary (fused): 503 layers, 31,586,006 parameters, 0 gradients, 169.8 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 9/9 [00:02<00:00,  3.09it/s]


                   all        133       1596      0.589       0.46      0.461      0.239
Speed: 0.2ms preprocess, 14.6ms inference, 0.0ms loss, 0.4ms postprocess per image
Results saved to /content/drive/MyDrive/Cartonomics/Cocass/runs/yolo/weighted_loss_2/weighted_loss_exp_254


COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     name                  : widespread_glaze_5027
COMET INFO:     url                   : https://www.comet.com/jiaming-jiang/cartonomics/6a246ebd3c1948cc8e18e7e5d8e5a6a9
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     lr/pg0 [201]               : (3.980000000000002e-05, 0.06782828282828283)
COMET INFO:     lr/pg1 [201]               : (3.980000000000002e-05, 0.00195049898989899)
COMET INFO:     lr/pg2 [201]               : (3.980000000000002e-05, 0.00195049898989899)
COMET INFO:     metrics/mAP50(B) [202]     : (0.02497, 0.46098)
COMET INFO:     metrics/mAP50-95(B) [202]  : (0.00631, 0.23850342225129184)
COMET INFO:     metrics/precision(B) [202] : (0.02103, 0.61203)

In [ ]:
# Here, comet is used to the logs during training and inspect if something went wrong. You can comment the following lines if you don't want to use comet (about experiment)
# The logs are also present in the runs folder
from comet_ml import Experiment
from comet_ml.integration.pytorch import log_model
torch.cuda.empty_cache()

experiment = Experiment(
  api_key="nHYScUpt7snJ6SYAs8tydMqcr",
  project_name="cartonomics",
  workspace="jiaming-jiang"
)

#Hyperparameters

results = detection_model.train(
    data=(HOME/"data/yolo_datasets/Detection/data.yaml").as_posix(),
    batch=8,  # Batch size
    epochs=100,  # Number of epochs
    close_mosaic=100,
    imgsz=640,  # Image size
    #optimizer='Adam',
    lr0=0.002,
    momentum=0.9,
    pretrained=True,
    project=(HOME/"runs/yolo/weighted_loss_2").as_posix(),  # Project directory
    name="weighted_loss_exp_2",
    cls=2.0,  # Increase the weight for classification loss
    patience=100,  # Early stopping patience
    verbose=False,  # Less verbose output
    cache=True,  # Cache dataset in memory
    amp=True,  # Enable mixed precision training
    #augment=False,
    #hyp=hyp_path.as_posix(),  # Pass the custom hyperparameters file
)

experiment.end()    #Do not forget to end the experiment to save the results in Comet
#this can take a while

COMET WARNING: To get all data logged automatically, import comet_ml before the following modules: sklearn, torch.
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/jiaming-jiang/cartonomics/32078f94de1d4f92b991b8625ee90f29



Ultralytics 8.3.32 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (NVIDIA L4, 22700MiB)
engine/trainer: task=detect, mode=train, model=/root/.cache/huggingface/hub/models--crh-ehess--yolass-1.0/snapshots/df7779d835e9f045e21f3a29c85c4bdd7dfa74d4/yolass_1-0.pt, data=/content/drive/MyDrive/Cartonomics/Cocass/data/yolo_datasets/Detection/data.yaml, epochs=100, time=None, patience=100, batch=8, imgsz=640, save=True, save_period=-1, cache=True, device=None, workers=8, project=/content/drive/MyDrive/Cartonomics/Cocass/runs/yolo/weighted_loss_2, name=weighted_loss_exp_255, exist_ok=False, pretrained=True, optimizer=Adam, verbose=False, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=100, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride

train: Scanning /content/drive/MyDrive/Cartonomics/Cocass/data/yolo_datasets/Detection/train/labels.cache... 572 images, 226 backgrounds, 50 corrupt: 100%|██████████| 572/572 [00:00<?, ?it/s]

train: WARNING ⚠️ /content/drive/MyDrive/Cartonomics/Cocass/data/yolo_datasets/Detection/train/images/3785973075.png: ignoring corrupt image/label: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (38,) + inhomogeneous part.
train: WARNING ⚠️ /content/drive/MyDrive/Cartonomics/Cocass/data/yolo_datasets/Detection/train/images/3786132335.png: ignoring corrupt image/label: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (71,) + inhomogeneous part.
train: WARNING ⚠️ /content/drive/MyDrive/Cartonomics/Cocass/data/yolo_datasets/Detection/train/images/3786135226.png: ignoring corrupt image/label: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (64,) + inhomogeneous part.
train: WARNING ⚠️ /content/drive/MyDrive/Cartonomics/Cocass/data/yolo_datase

WARNING ⚠️ cache='ram' may produce non-deterministic training results. Consider cache='disk' as a deterministic alternative if your disk space allows.


...

                   all        133       1596        0.6      0.481      0.475      0.241



100 epochs completed in 0.807 hours.
Optimizer stripped from /content/drive/MyDrive/Cartonomics/Cocass/runs/yolo/weighted_loss_2/weighted_loss_exp_255/weights/last.pt, 64.1MB
Optimizer stripped from /content/drive/MyDrive/Cartonomics/Cocass/runs/yolo/weighted_loss_2/weighted_loss_exp_255/weights/best.pt, 64.1MB

Validating /content/drive/MyDrive/Cartonomics/Cocass/runs/yolo/weighted_loss_2/weighted_loss_exp_255/weights/best.pt...
Ultralytics 8.3.32 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (NVIDIA L4, 22700MiB)
YOLOv10x summary (fused): 503 layers, 31,586,006 parameters, 0 gradients, 169.8 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 9/9 [00:03<00:00,  2.85it/s]


                   all        133       1596      0.605      0.481      0.479      0.246
Speed: 0.2ms preprocess, 14.0ms inference, 0.0ms loss, 0.4ms postprocess per image
Results saved to /content/drive/MyDrive/Cartonomics/Cocass/runs/yolo/weighted_loss_2/weighted_loss_exp_255


COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     name                  : frozen_marble_3436
COMET INFO:     url                   : https://www.comet.com/jiaming-jiang/cartonomics/32078f94de1d4f92b991b8625ee90f29
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     lr/pg0 [201]               : (3.980000000000002e-05, 0.06782828282828283)
COMET INFO:     lr/pg1 [201]               : (3.980000000000002e-05, 0.00195049898989899)
COMET INFO:     lr/pg2 [201]               : (3.980000000000002e-05, 0.00195049898989899)
COMET INFO:     metrics/mAP50(B) [202]     : (0.0632, 0.47908)
COMET INFO:     metrics/mAP50-95(B) [202]  : (0.03048, 0.24571475821492803)
COMET INFO:     metrics/precision(B) [202] : (0.21957, 0.61877)
COM

### Epoch100Batch8 Blanked Cropped 512 - SingleLabel

In [ ]:
# Here, comet is used to the logs during training and inspect if something went wrong. You can comment the following lines if you don't want to use comet (about experiment)
# The logs are also present in the runs folder
from comet_ml import Experiment
from comet_ml.integration.pytorch import log_model
torch.cuda.empty_cache()

experiment = Experiment(
  api_key="nHYScUpt7snJ6SYAs8tydMqcr",
  project_name="cartonomics",
  workspace="jiaming-jiang"
)

#Hyperparameters

results = detection_model.train(
    data=(HOME/"data/yolo_datasets/Detection/data.yaml").as_posix(),
    batch=8,  # Batch size
    epochs=100,  # Number of epochs
    close_mosaic=100,
    imgsz=512,  # Image size
    #optimizer='AdamW',
    lr0=0.002,
    momentum=0.9,
    pretrained=True,
    project=(HOME/"runs/yolo/weighted_loss_2").as_posix(),  # Project directory
    name="weighted_loss_exp_2",
    cls=2.0,  # Increase the weight for classification loss
    patience=100,  # Early stopping patience
    verbose=False,  # Less verbose output
    cache=True,  # Cache dataset in memory
    amp=True,  # Enable mixed precision training
    #augment=False,
    #hyp=hyp_path.as_posix(),  # Pass the custom hyperparameters file
)

experiment.end()    #Do not forget to end the experiment to save the results in Comet
#this can take a while

COMET WARNING: To get all data logged automatically, import comet_ml before the following modules: sklearn, torch.
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/jiaming-jiang/cartonomics/fc369e03a90f4156b60e623ed12d6323



Ultralytics 8.3.32 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (NVIDIA L4, 22700MiB)
engine/trainer: task=detect, mode=train, model=/root/.cache/huggingface/hub/models--crh-ehess--yolass-1.0/snapshots/df7779d835e9f045e21f3a29c85c4bdd7dfa74d4/yolass_1-0.pt, data=/content/drive/MyDrive/Cartonomics/Cocass/data/yolo_datasets/Detection/data.yaml, epochs=100, time=None, patience=100, batch=8, imgsz=512, save=True, save_period=-1, cache=True, device=None, workers=8, project=/content/drive/MyDrive/Cartonomics/Cocass/runs/yolo/weighted_loss_2, name=weighted_loss_exp_256, exist_ok=False, pretrained=True, optimizer=auto, verbose=False, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=100, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride

train: Scanning /content/drive/MyDrive/Cartonomics/Cocass/data/yolo_datasets/Detection/train/labels.cache... 572 images, 226 backgrounds, 50 corrupt: 100%|██████████| 572/572 [00:00<?, ?it/s]

train: WARNING ⚠️ /content/drive/MyDrive/Cartonomics/Cocass/data/yolo_datasets/Detection/train/images/3785973075.png: ignoring corrupt image/label: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (38,) + inhomogeneous part.
train: WARNING ⚠️ /content/drive/MyDrive/Cartonomics/Cocass/data/yolo_datasets/Detection/train/images/3786132335.png: ignoring corrupt image/label: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (71,) + inhomogeneous part.
train: WARNING ⚠️ /content/drive/MyDrive/Cartonomics/Cocass/data/yolo_datasets/Detection/train/images/3786135226.png: ignoring corrupt image/label: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (64,) + inhomogeneous part.
train: WARNING ⚠️ /content/drive/MyDrive/Cartonomics/Cocass/data/yolo_datase

WARNING ⚠️ cache='ram' may produce non-deterministic training results. Consider cache='disk' as a deterministic alternative if your disk space allows.


...

                   all        133       1596      0.557      0.427      0.438      0.214



100 epochs completed in 0.587 hours.
Optimizer stripped from /content/drive/MyDrive/Cartonomics/Cocass/runs/yolo/weighted_loss_2/weighted_loss_exp_256/weights/last.pt, 64.1MB
Optimizer stripped from /content/drive/MyDrive/Cartonomics/Cocass/runs/yolo/weighted_loss_2/weighted_loss_exp_256/weights/best.pt, 64.1MB

Validating /content/drive/MyDrive/Cartonomics/Cocass/runs/yolo/weighted_loss_2/weighted_loss_exp_256/weights/best.pt...
Ultralytics 8.3.32 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (NVIDIA L4, 22700MiB)
YOLOv10x summary (fused): 503 layers, 31,586,006 parameters, 0 gradients, 169.8 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 9/9 [00:02<00:00,  3.99it/s]


                   all        133       1596      0.557      0.427      0.438      0.214
Speed: 0.1ms preprocess, 9.8ms inference, 0.0ms loss, 0.1ms postprocess per image
Results saved to /content/drive/MyDrive/Cartonomics/Cocass/runs/yolo/weighted_loss_2/weighted_loss_exp_256


COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     name                  : selective_radius_2587
COMET INFO:     url                   : https://www.comet.com/jiaming-jiang/cartonomics/fc369e03a90f4156b60e623ed12d6323
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     lr/pg0 [201]               : (3.980000000000002e-05, 0.00195049898989899)
COMET INFO:     lr/pg1 [201]               : (3.980000000000002e-05, 0.00195049898989899)
COMET INFO:     lr/pg2 [201]               : (3.980000000000002e-05, 0.00195049898989899)
COMET INFO:     metrics/mAP50(B) [202]     : (0.00059, 0.437559896667857)
COMET INFO:     metrics/mAP50-95(B) [202]  : (0.00014, 0.21394)
COMET INFO:     metrics/precision(B) [202] : (0.00116, 0.59772)
C

### Epoch100Batch8 Blanked Crop_Fraction - SingleLabel

In [ ]:
# Here, comet is used to the logs during training and inspect if something went wrong. You can comment the following lines if you don't want to use comet (about experiment)
# The logs are also present in the runs folder
from comet_ml import Experiment
from comet_ml.integration.pytorch import log_model
torch.cuda.empty_cache()

experiment = Experiment(
  api_key="nHYScUpt7snJ6SYAs8tydMqcr",
  project_name="cartonomics",
  workspace="jiaming-jiang"
)

#Hyperparameters


results = detection_model.train(
    data=(HOME/"data/yolo_datasets/Detection/data.yaml").as_posix(),
    batch=8,  # Batch size
    epochs=10,  # Number of epochs
    close_mosaic=10,
    imgsz=768,  # Image size
    optimizer='AdamW',
    lr0=0.002,
    momentum=0.9,
    pretrained=True,
    project=(HOME/"runs/yolo/weighted_loss_2").as_posix(),  # Project directory
    name="crop_fraction",
    cls=2.0,  # Increase the weight for classification loss
    patience=100,  # Early stopping patience
    verbose=False,  # Less verbose output
    cache=True,  # Cache dataset in memory
    amp=True,  # Enable mixed precision training
    crop_fraction=0.7
    #augment=False,
    #hyp=hyp_path.as_posix(),  # Pass the custom hyperparameters file
)

experiment.end()    #Do not forget to end the experiment to save the results in Comet
#this can take a while

COMET WARNING: To get all data logged automatically, import comet_ml before the following modules: sklearn, torch.
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/jiaming-jiang/cartonomics/9ecba4c7354146fbb64d178f4c8e6cc8



Ultralytics 8.3.34 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (NVIDIA L4, 22700MiB)
engine/trainer: task=detect, mode=train, model=/root/.cache/huggingface/hub/models--crh-ehess--yolass-1.0/snapshots/df7779d835e9f045e21f3a29c85c4bdd7dfa74d4/yolass_1-0.pt, data=/content/drive/MyDrive/Cartonomics/Cocass/data/yolo_datasets/Detection/data.yaml, epochs=10, time=None, patience=100, batch=8, imgsz=768, save=True, save_period=-1, cache=True, device=None, workers=8, project=/content/drive/MyDrive/Cartonomics/Cocass/runs/yolo/weighted_loss_2, name=crop_fraction4, exist_ok=False, pretrained=True, optimizer=AdamW, verbose=False, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stre

train: Scanning /content/drive/MyDrive/Cartonomics/Cocass/data/yolo_datasets/Detection/train/labels.cache... 572 images, 226 backgrounds, 50 corrupt: 100%|██████████| 572/572 [00:00<?, ?it/s]

train: WARNING ⚠️ /content/drive/MyDrive/Cartonomics/Cocass/data/yolo_datasets/Detection/train/images/3785973075.png: ignoring corrupt image/label: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (38,) + inhomogeneous part.
train: WARNING ⚠️ /content/drive/MyDrive/Cartonomics/Cocass/data/yolo_datasets/Detection/train/images/3786132335.png: ignoring corrupt image/label: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (71,) + inhomogeneous part.
train: WARNING ⚠️ /content/drive/MyDrive/Cartonomics/Cocass/data/yolo_datasets/Detection/train/images/3786135226.png: ignoring corrupt image/label: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (64,) + inhomogeneous part.
train: WARNING ⚠️ /content/drive/MyDrive/Cartonomics/Cocass/data/yolo_datase

WARNING ⚠️ cache='ram' may produce non-deterministic training results. Consider cache='disk' as a deterministic alternative if your disk space allows.


...

                   all        133       1596      0.457      0.315      0.296      0.148



10 epochs completed in 0.128 hours.
Optimizer stripped from /content/drive/MyDrive/Cartonomics/Cocass/runs/yolo/weighted_loss_2/crop_fraction4/weights/last.pt, 64.1MB
Optimizer stripped from /content/drive/MyDrive/Cartonomics/Cocass/runs/yolo/weighted_loss_2/crop_fraction4/weights/best.pt, 64.1MB

Validating /content/drive/MyDrive/Cartonomics/Cocass/runs/yolo/weighted_loss_2/crop_fraction4/weights/best.pt...
Ultralytics 8.3.34 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (NVIDIA L4, 22700MiB)
YOLOv10x summary (fused): 503 layers, 31,586,006 parameters, 0 gradients, 169.8 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 9/9 [00:03<00:00,  2.70it/s]


                   all        133       1596      0.457      0.315      0.296      0.148
Speed: 0.3ms preprocess, 20.1ms inference, 0.0ms loss, 0.1ms postprocess per image
Results saved to /content/drive/MyDrive/Cartonomics/Cocass/runs/yolo/weighted_loss_2/crop_fraction4


COMET WARNING: Couldn't retrieve and log Google Colab notebook content, reason: 'NoneType' object is not subscriptable
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     name                  : successful_pie_3433
COMET INFO:     url                   : https://www.comet.com/jiaming-jiang/cartonomics/9ecba4c7354146fbb64d178f4c8e6cc8
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     lr/pg0 [21]               : (0.00021799999999999996, 0.06782828282828283)
COMET INFO:     lr/pg1 [21]               : (0.00021799999999999996, 0.0015958989898989901)
COMET INFO:     lr/pg2 [21]               : (0.00021799999999999996, 0.0015958989898989901)
COMET INFO:     metrics/mAP50(B) [22]     : (0.08606, 0.29626)
COMET INFO:     met

### Epoch100Batch8 Blanked Cropped 768 - SingleLabel

In [ ]:
# Here, comet is used to the logs during training and inspect if something went wrong. You can comment the following lines if you don't want to use comet (about experiment)
# The logs are also present in the runs folder
from comet_ml import Experiment
from comet_ml.integration.pytorch import log_model
torch.cuda.empty_cache()

experiment = Experiment(
  api_key="nHYScUpt7snJ6SYAs8tydMqcr",
  project_name="cartonomics",
  workspace="jiaming-jiang"
)

#Hyperparameters


results = detection_model.train(
    data=(HOME/"data/yolo_datasets/Detection/data.yaml").as_posix(),
    batch=8,  # Batch size
    epochs=100,  # Number of epochs
    close_mosaic=100,
    imgsz=768,  # Image size
    #optimizer='AdamW',
    lr0=0.002,
    momentum=0.9,
    pretrained=True,
    project=(HOME/"runs/yolo/weighted_loss_2").as_posix(),  # Project directory
    name="weighted_loss_exp_2",
    cls=2.0,  # Increase the weight for classification loss
    patience=100,  # Early stopping patience
    verbose=False,  # Less verbose output
    cache=True,  # Cache dataset in memory
    amp=True,  # Enable mixed precision training
    #augment=False,
    #hyp=hyp_path.as_posix(),  # Pass the custom hyperparameters file
)

experiment.end()    #Do not forget to end the experiment to save the results in Comet
#this can take a while

COMET WARNING: To get all data logged automatically, import comet_ml before the following modules: torch.
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/jiaming-jiang/cartonomics/13d3d92fb794492497f5f10369720125



Ultralytics 8.3.32 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (NVIDIA L4, 22700MiB)
engine/trainer: task=detect, mode=train, model=/root/.cache/huggingface/hub/models--crh-ehess--yolass-1.0/snapshots/df7779d835e9f045e21f3a29c85c4bdd7dfa74d4/yolass_1-0.pt, data=/content/drive/MyDrive/Cartonomics/Cocass/data/yolo_datasets/Detection/data.yaml, epochs=100, time=None, patience=100, batch=8, imgsz=768, save=True, save_period=-1, cache=True, device=None, workers=8, project=/content/drive/MyDrive/Cartonomics/Cocass/runs/yolo/weighted_loss_2, name=weighted_loss_exp_257, exist_ok=False, pretrained=True, optimizer=auto, verbose=False, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=100, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride

100%|██████████| 755k/755k [00:00<00:00, 90.3MB/s]


Overriding model.yaml nc=17 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1      2320  ultralytics.nn.modules.conv.Conv             [3, 80, 3, 2]                 
  1                  -1  1    115520  ultralytics.nn.modules.conv.Conv             [80, 160, 3, 2]               
  2                  -1  3    436800  ultralytics.nn.modules.block.C2f             [160, 160, 3, True]           
  3                  -1  1    461440  ultralytics.nn.modules.conv.Conv             [160, 320, 3, 2]              
  4                  -1  6   3281920  ultralytics.nn.modules.block.C2f             [320, 320, 6, True]           
  5                  -1  1    213120  ultralytics.nn.modules.block.SCDown          [320, 640, 3, 2]              
  6                  -1  6   4604160  ultralytics.nn.modules.block.C2fCIB          [640, 640, 6, True]           
  7                  -1  1    417920  ultralytics

train: Scanning /content/drive/MyDrive/Cartonomics/Cocass/data/yolo_datasets/Detection/train/labels.cache... 572 images, 226 backgrounds, 50 corrupt: 100%|██████████| 572/572 [00:00<?, ?it/s]

...

                   all        133       1596      0.565      0.486      0.464      0.236



100 epochs completed in 1.147 hours.
Optimizer stripped from /content/drive/MyDrive/Cartonomics/Cocass/runs/yolo/weighted_loss_2/weighted_loss_exp_257/weights/last.pt, 64.1MB
Optimizer stripped from /content/drive/MyDrive/Cartonomics/Cocass/runs/yolo/weighted_loss_2/weighted_loss_exp_257/weights/best.pt, 64.1MB

Validating /content/drive/MyDrive/Cartonomics/Cocass/runs/yolo/weighted_loss_2/weighted_loss_exp_257/weights/best.pt...
Ultralytics 8.3.32 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (NVIDIA L4, 22700MiB)
YOLOv10x summary (fused): 503 layers, 31,586,006 parameters, 0 gradients, 169.8 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 9/9 [00:03<00:00,  2.57it/s]


                   all        133       1596      0.606       0.49      0.476      0.242
Speed: 0.2ms preprocess, 21.0ms inference, 0.0ms loss, 0.9ms postprocess per image
Results saved to /content/drive/MyDrive/Cartonomics/Cocass/runs/yolo/weighted_loss_2/weighted_loss_exp_257


COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     name                  : selected_mandarine_2174
COMET INFO:     url                   : https://www.comet.com/jiaming-jiang/cartonomics/13d3d92fb794492497f5f10369720125
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     lr/pg0 [201]               : (3.980000000000002e-05, 0.00195049898989899)
COMET INFO:     lr/pg1 [201]               : (3.980000000000002e-05, 0.00195049898989899)
COMET INFO:     lr/pg2 [201]               : (3.980000000000002e-05, 0.00195049898989899)
COMET INFO:     metrics/mAP50(B) [202]     : (0.00279, 0.4764519838170911)
COMET INFO:     metrics/mAP50-95(B) [202]  : (0.00085, 0.24232)
COMET INFO:     metrics/precision(B) [202] : (0.00496, 0.61705

### Epoch100Batch8 Blanked Cropped 256 - SingleLabel

In [ ]:
# Here, comet is used to the logs during training and inspect if something went wrong. You can comment the following lines if you don't want to use comet (about experiment)
# The logs are also present in the runs folder
from comet_ml import Experiment
from comet_ml.integration.pytorch import log_model
torch.cuda.empty_cache()

experiment = Experiment(
  api_key="nHYScUpt7snJ6SYAs8tydMqcr",
  project_name="cartonomics",
  workspace="jiaming-jiang"
)

#Hyperparameters


results = detection_model.train(
    data=(HOME/"data/yolo_datasets/Detection/data.yaml").as_posix(),
    batch=8,  # Batch size
    epochs=100,  # Number of epochs
    close_mosaic=100,
    imgsz=256,  # Image size
    #optimizer='AdamW',
    lr0=0.002,
    momentum=0.9,
    pretrained=True,
    project=(HOME/"runs/yolo/weighted_loss_2").as_posix(),  # Project directory
    name="weighted_loss_exp_2",
    cls=2.0,  # Increase the weight for classification loss
    patience=100,  # Early stopping patience
    verbose=False,  # Less verbose output
    cache=True,  # Cache dataset in memory
    amp=True,  # Enable mixed precision training
    #augment=False,
    #hyp=hyp_path.as_posix(),  # Pass the custom hyperparameters file
)

experiment.end()    #Do not forget to end the experiment to save the results in Comet
#this can take a while

COMET WARNING: To get all data logged automatically, import comet_ml before the following modules: torch.
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/jiaming-jiang/cartonomics/9b3711478bb84ea2a94a7872ffbba707



Ultralytics 8.3.32 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (NVIDIA L4, 22700MiB)
engine/trainer: task=detect, mode=train, model=/root/.cache/huggingface/hub/models--crh-ehess--yolass-1.0/snapshots/df7779d835e9f045e21f3a29c85c4bdd7dfa74d4/yolass_1-0.pt, data=/content/drive/MyDrive/Cartonomics/Cocass/data/yolo_datasets/Detection/data.yaml, epochs=100, time=None, patience=100, batch=8, imgsz=256, save=True, save_period=-1, cache=True, device=None, workers=8, project=/content/drive/MyDrive/Cartonomics/Cocass/runs/yolo/weighted_loss_2, name=weighted_loss_exp_258, exist_ok=False, pretrained=True, optimizer=auto, verbose=False, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=100, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride

train: Scanning /content/drive/MyDrive/Cartonomics/Cocass/data/yolo_datasets/Detection/train/labels.cache... 572 images, 226 backgrounds, 50 corrupt: 100%|██████████| 572/572 [00:00<?, ?it/s]

train: WARNING ⚠️ /content/drive/MyDrive/Cartonomics/Cocass/data/yolo_datasets/Detection/train/images/3785973075.png: ignoring corrupt image/label: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (38,) + inhomogeneous part.
train: WARNING ⚠️ /content/drive/MyDrive/Cartonomics/Cocass/data/yolo_datasets/Detection/train/images/3786132335.png: ignoring corrupt image/label: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (71,) + inhomogeneous part.
train: WARNING ⚠️ /content/drive/MyDrive/Cartonomics/Cocass/data/yolo_datasets/Detection/train/images/3786135226.png: ignoring corrupt image/label: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (64,) + inhomogeneous part.
train: WARNING ⚠️ /content/drive/MyDrive/Cartonomics/Cocass/data/yolo_datase

WARNING ⚠️ cache='ram' may produce non-deterministic training results. Consider cache='disk' as a deterministic alternative if your disk space allows.


...

                   all        133       1596      0.414      0.171      0.164     0.0621



100 epochs completed in 0.411 hours.
Optimizer stripped from /content/drive/MyDrive/Cartonomics/Cocass/runs/yolo/weighted_loss_2/weighted_loss_exp_258/weights/last.pt, 64.0MB
Optimizer stripped from /content/drive/MyDrive/Cartonomics/Cocass/runs/yolo/weighted_loss_2/weighted_loss_exp_258/weights/best.pt, 64.0MB

Validating /content/drive/MyDrive/Cartonomics/Cocass/runs/yolo/weighted_loss_2/weighted_loss_exp_258/weights/best.pt...
Ultralytics 8.3.32 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (NVIDIA L4, 22700MiB)
YOLOv10x summary (fused): 503 layers, 31,586,006 parameters, 0 gradients, 169.8 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 9/9 [00:01<00:00,  7.95it/s]


                   all        133       1596      0.443      0.169      0.173     0.0655
Speed: 0.1ms preprocess, 6.2ms inference, 0.0ms loss, 0.1ms postprocess per image
Results saved to /content/drive/MyDrive/Cartonomics/Cocass/runs/yolo/weighted_loss_2/weighted_loss_exp_258


COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     name                  : contemporary_garden_6792
COMET INFO:     url                   : https://www.comet.com/jiaming-jiang/cartonomics/9b3711478bb84ea2a94a7872ffbba707
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     lr/pg0 [201]               : (3.980000000000002e-05, 0.00195049898989899)
COMET INFO:     lr/pg1 [201]               : (3.980000000000002e-05, 0.00195049898989899)
COMET INFO:     lr/pg2 [201]               : (3.980000000000002e-05, 0.00195049898989899)
COMET INFO:     metrics/mAP50(B) [202]     : (0.00022, 0.173449864391408)
COMET INFO:     metrics/mAP50-95(B) [202]  : (9e-05, 0.06547851468991386)
COMET INFO:     metrics/precision(B) [202] : (0.0004


### Epoch100Batch4 Blanked Cropped 768 - SingleLabel

In [ ]:
# Here, comet is used to the logs during training and inspect if something went wrong. You can comment the following lines if you don't want to use comet (about experiment)
# The logs are also present in the runs folder
from comet_ml import Experiment
from comet_ml.integration.pytorch import log_model
torch.cuda.empty_cache()

experiment = Experiment(
  api_key="nHYScUpt7snJ6SYAs8tydMqcr",
  project_name="cartonomics",
  workspace="jiaming-jiang"
)

#Hyperparameters


results = detection_model.train(
    data=(HOME/"data/yolo_datasets/Detection/data.yaml").as_posix(),
    batch=4,  # Batch size
    epochs=100,  # Number of epochs
    close_mosaic=100,
    imgsz=768,  # Image size
    optimizer='AdamW',
    lr0=0.00144,
    momentum=0.9,
    pretrained=True,
    project=(HOME/"runs/yolo/weighted_loss_2").as_posix(),  # Project directory
    name="weighted_loss_exp_2",
    cls=2.0,  # Increase the weight for classification loss
    patience=100,  # Early stopping patience
    verbose=False,  # Less verbose output
    cache=True,  # Cache dataset in memory
    amp=True,  # Enable mixed precision training
    #augment=False,
    #hyp=hyp_path.as_posix(),  # Pass the custom hyperparameters file
)

experiment.end()    #Do not forget to end the experiment to save the results in Comet
#this can take a while

COMET WARNING: To get all data logged automatically, import comet_ml before the following modules: sklearn, torch.
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/jiaming-jiang/cartonomics/687e0f5ce88941ec807dfabe5bd324d2



Ultralytics 8.3.34 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (NVIDIA L4, 22700MiB)
engine/trainer: task=detect, mode=train, model=/root/.cache/huggingface/hub/models--crh-ehess--yolass-1.0/snapshots/df7779d835e9f045e21f3a29c85c4bdd7dfa74d4/yolass_1-0.pt, data=/content/drive/MyDrive/Cartonomics/Cocass/data/yolo_datasets/Detection/data.yaml, epochs=100, time=None, patience=100, batch=4, imgsz=768, save=True, save_period=-1, cache=True, device=None, workers=8, project=/content/drive/MyDrive/Cartonomics/Cocass/runs/yolo/weighted_loss_2, name=weighted_loss_exp_261, exist_ok=False, pretrained=True, optimizer=AdamW, verbose=False, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=100, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_strid

100%|██████████| 755k/755k [00:00<00:00, 120MB/s]


Overriding model.yaml nc=17 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1      2320  ultralytics.nn.modules.conv.Conv             [3, 80, 3, 2]                 
  1                  -1  1    115520  ultralytics.nn.modules.conv.Conv             [80, 160, 3, 2]               
  2                  -1  3    436800  ultralytics.nn.modules.block.C2f             [160, 160, 3, True]           
  3                  -1  1    461440  ultralytics.nn.modules.conv.Conv             [160, 320, 3, 2]              
  4                  -1  6   3281920  ultralytics.nn.modules.block.C2f             [320, 320, 6, True]           
  5                  -1  1    213120  ultralytics.nn.modules.block.SCDown          [320, 640, 3, 2]              
  6                  -1  6   4604160  ultralytics.nn.modules.block.C2fCIB          [640, 640, 6, True]           
  7                  -1  1    417920  ultralytics

train: Scanning /content/drive/MyDrive/Cartonomics/Cocass/data/yolo_datasets/Detection/train/labels.cache... 572 images, 226 backgrounds, 50 corrupt: 100%|██████████| 572/572 [00:00<?, ?it/s]

...

                   all        133       1596       0.56      0.417      0.413      0.215



100 epochs completed in 1.095 hours.
Optimizer stripped from /content/drive/MyDrive/Cartonomics/Cocass/runs/yolo/weighted_loss_2/weighted_loss_exp_261/weights/last.pt, 64.1MB
Optimizer stripped from /content/drive/MyDrive/Cartonomics/Cocass/runs/yolo/weighted_loss_2/weighted_loss_exp_261/weights/best.pt, 64.1MB

Validating /content/drive/MyDrive/Cartonomics/Cocass/runs/yolo/weighted_loss_2/weighted_loss_exp_261/weights/best.pt...
Ultralytics 8.3.34 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (NVIDIA L4, 22700MiB)
YOLOv10x summary (fused): 503 layers, 31,586,006 parameters, 0 gradients, 169.8 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 17/17 [00:02<00:00,  6.01it/s]


                   all        133       1596       0.56      0.417      0.413      0.215
Speed: 0.3ms preprocess, 17.9ms inference, 0.0ms loss, 0.1ms postprocess per image
Results saved to /content/drive/MyDrive/Cartonomics/Cocass/runs/yolo/weighted_loss_2/weighted_loss_exp_261


COMET WARNING: Couldn't retrieve and log Google Colab notebook content, reason: 'NoneType' object is not subscriptable
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     name                  : initial_chart_2218
COMET INFO:     url                   : https://www.comet.com/jiaming-jiang/cartonomics/687e0f5ce88941ec807dfabe5bd324d2
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     lr/pg0 [201]               : (2.865600000000001e-05, 0.06739745547073792)
COMET INFO:     lr/pg1 [201]               : (2.865600000000001e-05, 0.001407896427480916)
COMET INFO:     lr/pg2 [201]               : (2.865600000000001e-05, 0.001407896427480916)
COMET INFO:     metrics/mAP50(B) [202]     : (0.08901, 0.41828)
COMET INFO:     metri

In [ ]:
from comet_ml import Experiment
from ultralytics import YOLO
from torchvision.utils import make_grid

# Initialize Comet Experiment
experiment = Experiment(
    api_key="nHYScUpt7snJ6SYAs8tydMqcr",
    project_name="cartonomics",
    workspace="jiaming-jiang"
)

class CustomYOLO(YOLO):
    def train(self, **kwargs):
        # Start the training process
        super().train(**kwargs)

        # Access the trainer
        trainer = self.trainer

        # Hook into the dataloader
        train_loader = trainer.train_loader  # Access training dataloader

        # Iterate through a single epoch to capture images
        for batch_idx, batch in enumerate(train_loader):
            images = batch['img']  # Images after augmentation
            if batch_idx % 10 == 0:  # Log every 10 batches
                self.log_images_to_comet(images, trainer.epoch, batch_idx)

    def log_images_to_comet(self, images, epoch, batch_idx):
    # Ensure images are detached and moved to CPU
      images = images.clone().detach().cpu()

    # Normalize to [0, 1] if values are outside this range
      if images.max() > 1.0:
        images = torch.clamp(images / 255.0, min=0, max=1)

    # Create a grid of images
      grid = make_grid(images[:4], nrow=2, normalize=True, scale_each=True)

    # Convert the grid to a NumPy array (HWC format)
      grid_np = grid.permute(1, 2, 0).numpy()

    # Log the image grid to Comet.ml
      experiment.log_image(
        grid_np,
        name=f"Augmented Images - Epoch {epoch}, Batch {batch_idx}"
      )




COMET WARNING: To get all data logged automatically, import comet_ml before the following modules: sklearn, torch.
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/jiaming-jiang/cartonomics/1a24903fd6514883b8d7c1f5960970b9



In [ ]:
# Instantiate the model with the custom class
model = CustomYOLO(model_path)

# Train the model
model.train(
    data=(HOME/"data/yolo_datasets/Detection/data.yaml").as_posix(),
    epochs=3,
    batch=4,
    imgsz=768,
    project="runs/yolo/weighted_loss_2",
    name="weighted_loss_exp_2",
    optimizer='AdamW',
    lr0=0.00144,
    momentum=0.9,
    pretrained=True,
    cache=True,
    amp=True
)

# End the Comet experiment
experiment.end()


Ultralytics 8.3.34 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (NVIDIA L4, 22700MiB)
engine/trainer: task=detect, mode=train, model=/root/.cache/huggingface/hub/models--crh-ehess--yolass-1.0/snapshots/df7779d835e9f045e21f3a29c85c4bdd7dfa74d4/yolass_1-0.pt, data=/content/drive/MyDrive/Cartonomics/Cocass/data/yolo_datasets/Detection/data.yaml, epochs=3, time=None, patience=100, batch=4, imgsz=768, save=True, save_period=-1, cache=True, device=None, workers=8, project=runs/yolo/weighted_loss_2, name=weighted_loss_exp_24, exist_ok=False, pretrained=True, optimizer=AdamW, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augm

train: Scanning /content/drive/MyDrive/Cartonomics/Cocass/data/yolo_datasets/Detection/train/labels.cache... 572 images, 226 backgrounds, 50 corrupt: 100%|██████████| 572/572 [00:00<?, ?it/s]

train: WARNING ⚠️ /content/drive/MyDrive/Cartonomics/Cocass/data/yolo_datasets/Detection/train/images/3785973075.png: ignoring corrupt image/label: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (38,) + inhomogeneous part.
train: WARNING ⚠️ /content/drive/MyDrive/Cartonomics/Cocass/data/yolo_datasets/Detection/train/images/3786132335.png: ignoring corrupt image/label: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (71,) + inhomogeneous part.
train: WARNING ⚠️ /content/drive/MyDrive/Cartonomics/Cocass/data/yolo_datasets/Detection/train/images/3786135226.png: ignoring corrupt image/label: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (64,) + inhomogeneous part.
train: WARNING ⚠️ /content/drive/MyDrive/Cartonomics/Cocass/data/yolo_datase

WARNING ⚠️ cache='ram' may produce non-deterministic training results. Consider cache='disk' as a deterministic alternative if your disk space allows.


train: Caching images (0.9GB RAM): 100%|██████████| 522/522 [00:02<00:00, 183.75it/s]

...

                   all        133       1596        0.3      0.209      0.177      0.078



3 epochs completed in 0.077 hours.
Optimizer stripped from runs/yolo/weighted_loss_2/weighted_loss_exp_24/weights/last.pt, 64.1MB
Optimizer stripped from runs/yolo/weighted_loss_2/weighted_loss_exp_24/weights/best.pt, 64.1MB

Validating runs/yolo/weighted_loss_2/weighted_loss_exp_24/weights/best.pt...
Ultralytics 8.3.34 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (NVIDIA L4, 22700MiB)
YOLOv10x summary (fused): 503 layers, 31,586,006 parameters, 0 gradients, 169.8 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 17/17 [00:02<00:00,  5.93it/s]


                   all        133       1596      0.399      0.244      0.211     0.0896
Speed: 0.4ms preprocess, 18.1ms inference, 0.0ms loss, 0.2ms postprocess per image
Results saved to runs/yolo/weighted_loss_2/weighted_loss_exp_24


COMET WARNING: Couldn't retrieve and log Google Colab notebook content, reason: 'NoneType' object is not subscriptable
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     name                  : shiny_swallow_215
COMET INFO:     url                   : https://www.comet.com/jiaming-jiang/cartonomics/1a24903fd6514883b8d7c1f5960970b9
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     lr/pg0 [7]               : (0.0007428071246819296, 0.06739745547073792)
COMET INFO:     lr/pg1 [7]               : (0.0004763358778625955, 0.000640745038167939)
COMET INFO:     lr/pg2 [7]               : (0.0004763358778625955, 0.000640745038167939)
COMET INFO:     metrics/mAP50(B) [8]     : (0.12326, 0.2118)
COMET INFO:     metrics/mAP50-9


### Epoch100Batch4 Blanked Cropped - SingleLabel

In [ ]:
# Here, comet is used to the logs during training and inspect if something went wrong. You can comment the following lines if you don't want to use comet (about experiment)
# The logs are also present in the runs folder
from comet_ml import Experiment
from comet_ml.integration.pytorch import log_model
torch.cuda.empty_cache()

#detection_model = YOLO("yolov10x.pt") # The base model is imported here

experiment = Experiment(
  api_key="nHYScUpt7snJ6SYAs8tydMqcr",
  project_name="cartonomics",
  workspace="jiaming-jiang"
)

print("A")

#Hyperparameters

results = detection_model.train(
    data=(HOME/"data/yolo_datasets/Detection/data.yaml").as_posix(),
    batch=4,  # Batch size
    epochs=100,  # Number of epochs
    imgsz=800,  # Image size
    pretrained=True,
    project=(HOME/"runs/yolo/weighted_loss_2").as_posix(),  # Project directory
    name="weighted_loss_exp_2",
    cls=2.0,  # Increase the weight for classification loss
    patience=100,  # Early stopping patience
    verbose=False,  # Less verbose output
    cache=True,  # Cache dataset in memory
    amp=True,  # Enable mixed precision training
    close_mosaic = 5,
    lr0 = 0.005,
    #auto_augment = get_random_crop_augmentation,
    #crop_fraction = 0.5
)

experiment.end()    #Do not forget to end the experiment to save the results in Comet
#this can take a while

COMET WARNING: To get all data logged automatically, import comet_ml before the following modules: sklearn, torch.
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/jiaming-jiang/cartonomics/1ce5240039cb4c639372e5caaa628944



A
Ultralytics 8.3.34 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (NVIDIA L4, 22700MiB)
engine/trainer: task=detect, mode=train, model=/root/.cache/huggingface/hub/models--crh-ehess--yolass-1.0/snapshots/df7779d835e9f045e21f3a29c85c4bdd7dfa74d4/yolass_1-0.pt, data=/content/drive/MyDrive/Cartonomics/Cocass/data/yolo_datasets/Detection/data.yaml, epochs=100, time=None, patience=100, batch=4, imgsz=800, save=True, save_period=-1, cache=True, device=None, workers=8, project=/content/drive/MyDrive/Cartonomics/Cocass/runs/yolo/weighted_loss_2, name=weighted_loss_exp_262, exist_ok=False, pretrained=True, optimizer=auto, verbose=False, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=5, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride

train: Scanning /content/drive/MyDrive/Cartonomics/Cocass/data/yolo_datasets/Detection/train/labels.cache... 572 images, 226 backgrounds, 50 corrupt: 100%|██████████| 572/572 [00:00<?, ?it/s]

train: WARNING ⚠️ /content/drive/MyDrive/Cartonomics/Cocass/data/yolo_datasets/Detection/train/images/3785973075.png: ignoring corrupt image/label: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (38,) + inhomogeneous part.
train: WARNING ⚠️ /content/drive/MyDrive/Cartonomics/Cocass/data/yolo_datasets/Detection/train/images/3786132335.png: ignoring corrupt image/label: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (71,) + inhomogeneous part.
train: WARNING ⚠️ /content/drive/MyDrive/Cartonomics/Cocass/data/yolo_datasets/Detection/train/images/3786135226.png: ignoring corrupt image/label: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (64,) + inhomogeneous part.
train: WARNING ⚠️ /content/drive/MyDrive/Cartonomics/Cocass/data/yolo_datase

WARNING ⚠️ cache='ram' may produce non-deterministic training results. Consider cache='disk' as a deterministic alternative if your disk space allows.


...

     34/100      8.05G      3.137      7.587      1.947          8        800: 100%|██████████| 131/131 [00:37<00:00,  3.46it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 17/17 [00:03<00:00,  4.80it/s]

                   all        133       1596        0.5      0.353      0.355      0.168



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     35/100      7.97G      3.114      7.671      1.992         63        800:  79%|███████▉  | 104/131 [00:30<00:07,  3.43it/s]


KeyboardInterrupt: 

### Epoch12 Not Blanked Not Cropped - MultiLabel

In [ ]:
# Import Comet and other necessary libraries
from comet_ml import Experiment
import torch
torch.cuda.empty_cache()

# Define hyperparameters for the experiment
epochs = 30
imgsz = 640
cls_weight = 1.0  # Fixed classification loss weight
patience = 2
amp = True
cache = True

# List of batch sizes to experiment with
batch_size_list = [1, 2, 4, 8]

# Loop over each batch size for the experiment
for batch in batch_size_list:

    # Create a new experiment in Comet for each batch size
    experiment = Experiment(
        api_key="nHYScUpt7snJ6SYAs8tydMqcr",
        project_name="cartonomics",
        workspace="jiaming-jiang"
    )

    # Set a custom name for the experiment
    experiment.set_name(f"batch_size_exp_{batch}")

    # Log hyperparameters specific to this run
    experiment.log_parameters({
        "epochs": epochs,
        "imgsz": imgsz,
        "batch_size": batch,
        "cls_weight": cls_weight,
        "patience": patience,
        "amp": amp,
        "cache": cache
    })

    # Train the model with the current batch size
    results = detection_model.train(
        data=(HOME / "data/yolo_datasets/train/data.yaml").as_posix(),  # Path to the dataset
        batch=batch,  # Batch size (variable being experimented on)
        epochs=epochs,  # Fixed number of epochs
        imgsz=imgsz,  # Fixed image size
        plots=True,  # Enable plots
        pretrained=True,  # Use pretrained weights
        project=(HOME / "runs/yolo/baseline").as_posix(),  # Project directory
        name=f"batch_size_exp_{batch}",  # Naming each run based on batch size
        save_dir="train",  # Directory to save weights and logs
        patience=patience,  # Early stopping patience
        verbose=False,  # Disable verbose output
        cache=cache,  # Caching images in memory for speed
        amp=amp,  # Enable automatic mixed precision
        cls=cls_weight  # Classification loss weight
    )

    # End the Comet experiment to save the results
    experiment.end()

    # Print confirmation after each run
    print(f"Finished experiment with batch size: {batch}")


COMET WARNING: To get all data logged automatically, import comet_ml before the following modules: torch, sklearn.
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/jiaming-jiang/cartonomics/d6ec34355e784e959cc54a6c1a7087c3



Ultralytics 8.3.7 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=/root/.cache/huggingface/hub/models--crh-ehess--yolass-1.0/snapshots/df7779d835e9f045e21f3a29c85c4bdd7dfa74d4/yolass_1-0.pt, data=/content/drive/MyDrive/Cartonomics/Cocass/data/yolo_datasets/train/data.yaml, epochs=30, time=None, patience=2, batch=8, imgsz=640, save=True, save_period=-1, cache=True, device=None, workers=8, project=/content/drive/MyDrive/Cartonomics/Cocass/runs/yolo/baseline, name=batch_size_exp_82, exist_ok=False, pretrained=True, optimizer=auto, verbose=False, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=Fal

100%|██████████| 755k/755k [00:00<00:00, 42.7MB/s]


Overriding model.yaml nc=17 with nc=13

                   from  n    params  module                                       arguments                     
  0                  -1  1      2320  ultralytics.nn.modules.conv.Conv             [3, 80, 3, 2]                 
  1                  -1  1    115520  ultralytics.nn.modules.conv.Conv             [80, 160, 3, 2]               
  2                  -1  3    436800  ultralytics.nn.modules.block.C2f             [160, 160, 3, True]           
  3                  -1  1    461440  ultralytics.nn.modules.conv.Conv             [160, 320, 3, 2]              
  4                  -1  6   3281920  ultralytics.nn.modules.block.C2f             [320, 320, 6, True]           
  5                  -1  1    213120  ultralytics.nn.modules.block.SCDown          [320, 640, 3, 2]              
  6                  -1  6   4604160  ultralytics.nn.modules.block.C2fCIB          [640, 640, 6, True]           
  7                  -1  1    417920  ultralytic

train: Scanning /content/drive/MyDrive/Cartonomics/Cocass/data/yolo_datasets/train/train/labels.cache... 337 images, 0 backgrounds, 1 corrupt: 100%|██████████| 337/337 [00:00<?, ?it/s]

...

WARNING ⚠️ cache='ram' may produce non-deterministic training results. Consider cache='disk' as a deterministic alternative if your disk space allows.


val: Caching images (0.1GB RAM): 100%|██████████| 85/85 [00:00<00:00, 138.25it/s]


Plotting labels to /content/drive/MyDrive/Cartonomics/Cocass/runs/yolo/baseline/batch_size_exp_16/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000588, momentum=0.9) with parameter groups 185 weight(decay=0.0), 198 weight(decay=0.0005), 197 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to /content/drive/MyDrive/Cartonomics/Cocass/runs/yolo/baseline/batch_size_exp_16
Starting training for 30 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


  0%|          | 0/21 [00:00<?, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 64.00 MiB. GPU 0 has a total capacity of 14.75 GiB of which 53.06 MiB is free. Process 76061 has 14.69 GiB memory in use. Of the allocated memory 14.25 GiB is allocated by PyTorch, and 245.05 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

You can also inspect the logs with tensorboard

In [ ]:
!tensorboard --logdir runs/yolo/